In [ ]:
from Visualizers.labellednn import NeuralNetworkPlot
from IPython.display import IFrame, Image, display
from Visualizers.threeDConstructor import NeuralNetworkVisualization
from Visualizers.singleMinima import GradientDescentSimulator
from Visualizers.multiMinima import MultiMinimaSimulator
from Visualizers.OptimizerComparisons import MultiMinimaSimulatorAdaptive
from Visualizers.spinningPerceptron import PerceptronPlotlyDemo
from Visualizers.derivativeGraph import PlotlySurfaceWithGradient
import numpy as np
import pandas as pd


# **A Comprehensive Breakdown of Neural Networks with Reasoning Behind My Own Design Choices for my Neural Network**

Author: Patrick Erickson

---

## Abstract
---

Neural Networks have been gaining a lot of traction in the world as of recent, with the rise of giant models such as Chat-GPT and other LLMs. While the complexity of these giant AI models can not be explained by neural networks alone, they make up a core part of what makes them function. Furthermore, there are thousands of other types of models that use neural networks in their architecture to help them make predictions. In this document, I will be detailing the reason and intuition behind neural networks, the math behind them, including how to forward pass, activation functions, regularization techniques, and backpropagation (if you don't know what these mean, don't worry. We'll cover it!), and more complex stuff such as optimization functions. Lastly, I will go over an in-depth analysis on my own Neural Network's architecture so you can see my own intuitions behind the construction of one, from scratch without the use of popular libraries such as TensorFlow, PyTorch, and JAX. This article does assume, however, that you have a basic understanding of classification and regression, derivatives, and the derivative chain rule. I also expect that you understand how matrix multiplication, addition, and subtraction works. For the scope of this document, we will not be delving into Convolutional Heads or Time Series. I will also not be delving into Maximum Likelihood Estimators, but you should have a basic grasp of at least some probability distributions, such as gaussian and binomial.

### A Brief History of Neural Networks: The Perceptron
The birth of the Neural Network had much humbler beginnings than the giant models we see today. [Frank Rosenblatt](https://en.wikipedia.org/wiki/Frank_Rosenblatt), at the Cornell Aeronautical Laboratory, had constructed something called a single layer perceptron simulated on an IBM 704 in 1957. This "perceptron" was made to simulate an individual neuron within the human brain, which either fires or doesn't fire based on the input it's given. 

For example, let's say we know the following about someone:
  - Income
  - Properties Owned

and we we want to predict if that person is rich or not rich (we will use 0 for not rich and 1 for rich). Our perceptron will look as such:

In [ ]:
perceptronPlot = NeuralNetworkPlot(
    title="Perceptron",
    nodes_per_layer=[2,1,1],
    label_arrows=True,
    arrow_label_overrides={(0,0,0):"Income",(0,1,0):"Properties Owned",(1,0,0):"Are they Rich? (0/1)"}
)
perceptronPlot.show()

Then the intuition behind this is that there is some correlation between the sizes of the features and the final output of being rich or not. Fer example, the income and the number of properties owned aren't over a certain threshold, we would print 0, for being not rich. Likewise, if income and and property value aren over this threshold, we then predict 1 (rich). More specifically, the algorithm looks at all of the points it misclassifies and then performs an operation to move the hyperplane to fix that misclassification. This algorithm continues to do this until all points are satisfied. We can visualize how the perceptron works with the following simulation:

In [ ]:
perfectSep = PerceptronPlotlyDemo(
    title="Perceptron Simulation",
    n_points=100,
    max_updates=50,
    seed=42,
    linearlySeperable=0
)
perfectSep()

### Limitations of the Perceptron
As you can see, we slowly "fixed" the classifying line until all the "Rich" points are on one side of the line while all of the "Not Rich" points are on the other side of the line. We can therefore "linearly seperate" the data to make our predictions, which is the whole basis on how perceptrons work. However, what if the data isn't linearly seperable? For example, there could be some people who are considered "rich" who may only have one property, but that one property is a skyscraper in Manhattan worth billions. Our perceptron doesn't have this information, and we may not necessarily know this information to be able to put it into the perceptron. This "unexplained data" may cause our dataset to look more like this simulation, and is more analagous to the real world:

In [ ]:
perfectSep = PerceptronPlotlyDemo(
    title="Perceptron Simulation For non Linearly-Seperable Data",
    n_points=100,
    max_updates=200,
    seed=42,
    linearlySeperable=1
)
perfectSep()

The perceptron will have trouble classifying it, and the algorithm will never stop (converge), since there will always be some extra "movement" that can be made to try and fix the errors in the loss function. This issue only gets worse for datasets where linear seperability is borderline impossible. However, we can see that the data is still approximately linearly seperable, which allows us to be able to use methods such as logistic regression and other machine learning techniques will do a pretty good job of predicting the overall overall dataset, especially in the case we just described. But what about for data that isn't even remotely seperable? Let's look at the following example:

### Further Limitations of Perceptron with XOR

Assume we want to predict whether a customer would buy a product based on whether or not they visit the online and physical store of a specific company to review a product. It can be said that if a customer does not visit the online or real store, then the customer will not buy the product, because they are not interested. Likewise, a customer will most likely not walk out with anything if they visit both the online and real store, because they are currently assessing their options and they will not make a hasty decision on buying the product. However, if a customer visits the online store but not the real store, the customer may be more likely to buy it because they may think it's cheap and convenient. Likewise, a customer might be more enticed to buy a product if they physically saw it at the real store, but did not think to compare online, since the enticing prospect of buying and immediately having it is really high. We can model this problem with the following table:

In [ ]:
XOR = np.empty((4,2), dtype='U10')
XOR[0,0] = "No"
XOR[0,1] = "No"
XOR[1,0] = "No"
XOR[1,1] = "Yes"
XOR[2,0] = "Yes"
XOR[2,1] = "No"
XOR[3,0] = "Yes"
XOR[3,1] = "Yes"

labels = np.array(["No","Yes","Yes","No"])
XOR = np.c_[XOR,labels]
columns = ["Visted Store", "Visted Online", "Bought Item"]
df = pd.DataFrame(XOR,columns=columns)
df.head()

This is an example of a classic XOR problem, and even logistic regression and other classification methods perform poorly on this kind of data without any explicit feature engineering. Our perceptron will also suffer the same fate. The following is a simulation of different datapoints based on the company predictions:

In [ ]:
xorProblem = PerceptronPlotlyDemo(
    title="Perceptron Simulation For non Linearly-Seperable Data",
    n_points=100,
    max_updates=200,
    redName="Didn't Buy Item",
    blueName="Bought Item",
    seed=42,
    linearlySeperable=2
)
xorProblem()

As we expected, the perceptron performs extremely poorly. This phenomenon itself led to one of the biggest AI winters ever since the conception of the perceptron, slowing AI research to a near standstill for 20 years. However, this obviously does not pertain to today, as we have figured out a clever workaround for this issue. This, along with stochastic nature of the perceptron algorithm, did not specify loss efficiently, and would therefore oscillate with no real solution. In Layman's terms, the perceptron had no "confidence" in its decisions, based on the distance a point was from the classifying boundary.

## Introduction to Neural Networks
---


The Universal Approximation Theorem ultimately states that combining multiple neurons with some nonlinear activation can approximate any function, given enough of these neurons. This groundbreaking revelation in 1989 spurred the idea of stacking and aggregating inputs together into a single "layer" to effectively classify non-linear decision boundaries, such as the XOR problem described earlier. The introduction of the "confidence method" as an output mechanism further addressed the issue of perceptrons oscillating without convergence. This new approach, which focused on minimizing misclassification, required gradient descent optimization. Around the same period, a crucial algorithm called backpropagation was developed. Backpropagation efficiently computed chained derivatives through the network's hidden layers, enabling gradient descent to effectively tune perceptron weights, thereby minimizing confidence-based misclassifications, or in other words, reducing loss. Finally, breakthroughs in differentiable functions—known as activation functions—replaced the non-differentiable, step-based activation previously used by perceptrons (-1 for incorrect classification, +1 for correct). This shift to differentiable activations was the final essential component that allowed neural networks to be trained via gradient descent. Together, these advancements established the foundational framework of modern Artificial Neural Networks.

### How Neural Networks Learn
Neural Networks learn by creating predictions, then using that prediction and seeing how far it is away from the actualy associated value, often called the true label. These losses are aggregated, and then used to update the weights until the model can predict everything extremely well! Therefore, we break down neural network training into 3 problems:
 - Forward pass
 - Minimizing an objective function through backpropagation
 - evaluating its effectiveness with new data

### Types of activation functions

 Some of the activation functions mentioned are: 
   - ReLU (Rectified Linear Unit), a function x that is 0 when $x\leq0$
   - $\tanh$, used to create some value in between -1 and 1
   - sigmoid, usually used to model some probability, giving a value of 0 to 1.

 They can be defined as the following, with their derivatives:

$$
\operatorname{ReLU}(x) = \begin{cases}
x, & \text{if } x \geq 0, \\
0, & \text{if } x < 0.
\end{cases}
$$
$$
\operatorname{ReLU'}(x) = \begin{cases}
1, & \text{if } x \geq 0, \\
0, & \text{if } x < 0.
\end{cases}
$$
$$
\tanh(x) = \frac{e^x-e^{-x}}{e^x+e^{-x}} 
$$
$$
\tanh'(x) = 1 - (\frac{e^x-e^{-x}}{e^x+e^{-x}})^2 = 1 - \tanh^2(x)
$$
$$
\operatorname{sigmoid}(x) = \frac{1}{1+e^{-x}} \text{. Note that this is commonly denoted as } \sigma(x)
$$
$$
\operatorname{sigmoid}'(x) = 1 - \frac{e^{-x}}{(1+e^{-x})^2} = (\frac{1}{1+e^{-x}})(1-\frac{1}{1+e^{-x}}) = \sigma(x)(1-\sigma(x))
$$
  - **Note:** For sigmoid, due to the nature of the loss function it is usually good practice to ensure that values never go past ($1*10^{-8},1-1*10^{-8}$). This is called clipping, and ensures your super confident features don't diverge to infinity.

All of these can be used to replace the perceptron loss. **However, do note that for stable convergence it is highly recommended that you only use one type of activation and it's respective derivative for your entire network. This guarantees better convergence, and I tested this. Mixing activations just suck, you can try it yourself with my own [custom-implemented neural network](https://github.com/PatrickErickson4/FullyModularNumpyArtificialNeuralNetwork).** There are also many other types of activations, but for the sake of this demonstration, we will be working with these activations.

  - **Fun Fact:** Something interesting of note is that ReLU is piecewise, so you might be wondering how this is differentiable for backpropagation. Well, the secret lies in being able to see which features were activated on unactivated by relu. Since we define that if $x=0$, then the derivative will also be zero, we fix the nondifferentiability issue. Secondly, x will just be 1, which preserves information. Meanwhile the step function is either -1 or 1, with nothing with respect to x. This gives us 0 for the entire function, giving us no meaningful information for backpropagation.

### Loss Functions

There are also loss functions that assign some sort of non-discrete confidence of each prediction, in order to continue with the theme of differentiability for backpropagation. These are found at the end of the Neural Networks, commonly referred to as the "head". It is common to use softmax for classification problems. Softmax assignes a probability of confidence to each of the different categories for our problem, which we can then pick the highest in order to classify our problem. For example, if we have 3 categories, those being good, neutral, or bad, the number 1 will be split across all 3 of these categories based on the confidence for each prediction. There is also Mean Squared Error (MSE) for regression, which is the exact same formula used in that of linear regression. The following loss functions have the respective formulas, derivatives, and a reported loss, generally shown to the user when training:

**Classification:**
$$softmax \text{ for a single sample } p_i = \frac{e^{x_i}}{\sum_{j=1, i\in K}^Ke^{x_j}}. $$
In other words, the probability of every category for 1 sample in the dataset, and $p_i$ is analagous to the softmax function.
$$ \text{categorical cross entropy loss }= -\frac{1}{n}\sum_{i=1}^n \hat{y_i}\log(p_i)\text{. }$$ 
In other words, the amount the prediction deviated from the actual for the entire dataset.
$$softmax' \text{ for a single sample } = (p_i-\hat{y_i})\text{. }$$
In other words, how much we need to change our prediction by to fix the error.


**Regression:**
$$MSE \text{ prediction for a single sample }= \hat{y_i}\text{. }$$
In other words, the square of how much one variable is on the y axis away from our predicted best fit line.
$$MSE \text{ loss }= -\frac{1}{n}\sum_{i=1}^n (\hat{y_i}-\bar{y_i})^2\text{. }$$ 
In other words, the amount the prediction deviated from the actual for the entire dataset.
$$MSE' \text{ for a single sample }= \hat{y_i}-\bar{y_i}\text{. }$$
In other words, how much we need to change our prediction by to fix the error.

the following is a key for all of the variables:
  - $p_i$ is the softmax function
  - $\hat{y_i}$ is the true value that we are trying to predict for the label $i$ in our dataset.
  - $\bar{y_i}$ is the value we predicted with our model
  - $x_i$ are the values we get from the last layer.
  - $\frac{1}{n}\sum_{i=1}^n$ means the "mean" of the values.

For calculating the loss, it is much easier to think of it for every sample. We can calculate these then average the losses across all samples to get the middle loss value, which is generally shown to the client training a neural network. These equations are all based in fundamental statistics and the log likelihood to be able to find this total versus expected loss. Like the activation functions, there are also many different kinds of loss functions, especially for regression, but for the time being we will be sticking with these, as they are the most common. While you have most likely heard of mean squared error, you may not have heard of softmax. Let me explain it briefly:

#### What is softmax?
**Softmax turns all of your categories into probabilities of being picked.** Suppose you want to predict 10 different numbers, 0 through 9, from a dataset of images like the [MNIST Handwritten Digits Dataset](http://yann.lecun.com/exdb/mnist/). Let's say we want to predict the number 3. The idea is that the number produced for our model for the category of 3 will be a lot higher than the rest of the categories. What softmax does is it takes into account the values going into that specific category and divides it by the values of all the other categories to get a probability distribution for each category. If we look at the example, we can calculate the probability of us correctly picking 3 as such:
$$\frac{e^{x_3}}{e^{x_0}+e^{x_1}+e^{x_2}+e^{x_3}+e^{x_4}+e^{x_5}+e^{x_6}+e^{x_7}+e^{x_8}+e^{x_9}}$$
notice that this is the same as softmax we described as above, but expanded for the scope of our problem:
$$p_3 = \frac{e^{x_3}}{\sum_{i=0, 3\in \text{ the set of integers from 0-9 }}^9e^{x_i}}$$
In other words, $K$ represents all categories, and each x will be the corresponding specific value of the category, where the bigger number for a specific category yields a higher probability. If our model were correct, our probability distribution would look something like this.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(100)
base_logits = np.array([8, 8, 8, 10, 3, 7, 8, 2, 9, 8])
noise = np.random.uniform(-0.5, 0.5, size=base_logits.shape)
noisy_logits = base_logits + noise
if np.max(np.delete(noisy_logits, 3)) >= noisy_logits[3]:
    noisy_logits[3] = np.max(np.delete(noisy_logits, 3)) + 0.1
exp_logits = np.exp(noisy_logits)
probs = exp_logits / np.sum(exp_logits)
digits = np.arange(10)
colors = ['red' if i == 3 else 'skyblue' for i in range(10)]
fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.barh(digits, probs, color=colors)
for bar, prob in zip(bars, probs):
    width = bar.get_width()
    ax.text(width + 0.005, bar.get_y() + bar.get_height()/2, f'{prob:.3f}', va='center')

ax.set_yticks(digits)
ax.set_yticklabels(digits)
ax.set_title('Probability distribution for 10 Categories')
plt.figtext(0.5, 0.0, '3 has the highest probability, so we pick 3.', color='red', ha='center', fontsize=12)

plt.tight_layout()
plt.show()


**Notice how if we sum up the probabilities of all of the categories, we get 1.**

We can see that the model is "this confident" in predicting a 3. If certain probabilities are closer, that means our model is less confident in our prediction. Using this, we can classify many different categories. Modern architectures sometimes have thousands of these categories for image recognition, in order to accomplish things like object detection.

If instead all of these equations may look complicated, I promise it is as easy as just putting the function in the correct part of the formula. The derivations are all here for reference.

## Application of Neural Networks
---

Going back to our [**original shopping problem**](###Further-Limitations-of-Perceptron-with-XOR),  assume the following features are going to the shop ($x_1$) in store or online or not ($x_2$). We are going to simulate the following neural network, in order to see how a perceptron with a softmax head will perform.

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 1, 2, 2],
    label_arrows=False,
    arrow_label_overrides={}  
)
nn_plot.show()

We can see that this is almost exactly like the perceptron, but now we have a softmax output. **For the purpose of our example, note that there is an extra node that does not have any weights tied to it: it simply performs softmax**. When you are ready, press play on the following simulation, and see how the decision boundary changes on the graph. Stop it when the graph no longer seems to change.

In [ ]:
IFrame(src="https://playground.tensorflow.org/#activation=relu&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=1&seed=0.58130&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false",width=800,height=600)

 Based on the tensorflow playground, it is still impossible to linearly seperate the data. Let's try a different approach.  Using the Universal Approximation Theorem, we will construct the following neural network with 4 nodes in the hidden layer:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={}  # provide any overrides as 
)

# Display the plot.
nn_plot.show()

Let's see what happens when we try our simulation:

In [ ]:
IFrame(src='https://playground.tensorflow.org/#activation=relu&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=4&seed=0.58130&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false',width=800,height=600)

Notice how with four different linear combinators, we can create four different "segments" and finally solve the issue that has plagued AI researchers during the 70's and 80's.





### The need for layers

While the Universal Approximation theorem states that you can approximate any function $f(x)$ given the appropriate amount of nodes, this does not necessarily mean we should make  a single layer with this sufficient amount of nodes. By breaking each number of nodes up into seperate layers, we are able to compound on the features learned by each layer to arrived at a better gerneralization. We can see this motivation with the following architecture:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 8, 2, 2],
    label_arrows=False,
    arrow_label_overrides={}
)

# Display the plot.
nn_plot.show()

We can see this represented in Tensorflow. We will try to classify a spiral dataset:

In [ ]:
IFrame(src='https://playground.tensorflow.org/#activation=relu&regularization=L2&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.1&regularizationRate=0.003&noise=0&networkShape=8&seed=0.30700&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false',width=800,height=600)

By the example, you can see the model struggle to correctly seperate the feature space. However, let's define a different architecture:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 6,6, 2, 2],
    label_arrows=False,
    arrow_label_overrides={}
)

# Display the plot.
nn_plot.show()

Let's see how this performs in the playground on the same dataset.

In [ ]:
IFrame(src='https://playground.tensorflow.org/#activation=relu&regularization=L2&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.1&regularizationRate=0.003&noise=0&networkShape=6,6&seed=0.30700&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false',width=800,height=600)

You can see that the first layer learns bigger, more simple features, which are then fed into the second layer, where features are broken down and refined. As a result, we get a much better generalization than the 8 node example we had shown previously. This is the core idea behind deep learning: we can approximate almost any function, given enough layers and nodes. We have since learned how to numerically represent words and pictures, and we can therefore generalize even more complex ideas such as this. LLM's like ChatGPT and image generators such as DALL-E build off of this.

  - **Fun Fact:** There is a theory in statistics that states that the best models are the simplest, referred to as Ockham's Razor. Neural Networks are thought to have been unscalable because of this. However, do you see how some of these nodes you see how some the nodes aren't used at all? The idea behind sparsification: that a neural network only trains the nodes it needs to, and you can effectively prune these other nodes. The idea behind it therefore is that the more nodes you add, the more likely you are to find a solution that is really good at approximating something, because there are more routes that weights can travel through. Despite this, your model can still overfit if you have too many nodes, so keep that in mind.

## Thinking in Matrices: The Math Behind These Networks
---

Knowing the intuition behind why these neural networks are used, we can finally delve into the math that makes them work. Based on all of the connections for the neural network, we can see that each weight, being fully connected, can form a matrix of values. Let's look at out previous example, focusing on the blue arrow highlighting the area of interest:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1
)

# Display the plot.
nn_plot.show()

We can see that $x_1$ connects to all four nodes, as does $x_2$. This means that there will be 8 total arrows, which matches with our diagram. Each one of these white arrows for represent some weight to multiply our input from the previous layer by. The corresponding weight matrix can be represented as such:

$$
\left( \begin{array}{cccc}
w_{1,1}^{(1)} & w_{1,2}^{(1)} \\[0.5em]
w_{2,1}^{(1)} & w_{2,2}^{(1)} \\[0.5em]
w_{3,1}^{(1)} & w_{3,2}^{(1)} \\[0.5em]
w_{4,1}^{(1)} & w_{4,2}^{(1)} \\[0.5em]
\end{array} \right)
$$

Where every weight has the following properties:
$$w_{i\text{ , }j}^{(\ell)} $$

Think of this term as any single arrow in your neural network, where the subscripts and superscripts tell us which arrow it is. Each arrow will have some weight, which will be some number.
  - $w$ simply means weight number (the value we multiply our previous inputs by
  - $i$ specifies which node in the layer it is going to 
  - $j$ specifies which node from the previous layer the weight is coming from
  - $\ell$ specifies the layer we are trying to calculate the weights for. For example, $\ell=1$ for this particular weight matrix.
  - if we drop the subscript $j$, that means we are talking about a particular node in the layer
  - if we drop both subscripts and just have a capital $W^{(\ell)}$, this means we are talking about all arrows pointing into the LAYER, which means the **entire weight matrix**.
  - Any Weight Matrix feeding into a layer will have the dimensions (nodes of the current layer) x (nodes/features of the previous layer). In other words, $dim(W^{(\ell)}) = \ell\text{ x }(\ell-1)$

Up until now, we have also been simplifying the neural network for visualization purposes. In reality, Neural Networks have a bias for every layer that looks something like this:



In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    bias=True
)

# Display the plot.
nn_plot.show()

The reason for this bias term is to ensure that some weights do not get too big to overcompensate for a poor intercept, when weights might have learned the shape already. You can imagine this with our spiral example. Imagine our weights learned a really good boundary for classifying the weight, but the whole boundary is shifted to the right. The bias accounts for this and shifts it back to its correct place. This bias term can be represented as
$$b^{(\ell)}_i$$
where 
  - i is the node the bias value corresponds to
  - $\ell$ is the layer the bias is being used to calculate.
  - $b^{(\ell)}$ is the bias value for that entire layer
  - $b_i^{(\ell)}$ is the bias value for the $i^{th}$ node of the $\ell^{th}$ layer
so an individual bias for the layer we are talking about for node can be represented as
$$b_1^{(1)}$$
which will correspond to some single scalar number. Likewise, the bias for the entire layer we are pointing at can be represented as:
$$
b^{(1)} = 
\left( \begin{array}{cccc}
b_{1}^{(1)} \\[0.5em]
b_{2}^{(1)} \\[0.5em]
b_{3}^{(1)} \\[0.5em]
b_{4}^{(1)} \\[0.5em]
\end{array} \right)
$$
where each subscript represents the node the bias corresponds to.

## Predicting a Number: Forward Pass
---

By computing forward pass, we get a predicted value, given all of the features we are predicting with in our dataset. The final output will give what the neural networks "thinks" the value should be. 


Our forward pass can be though of in 4 steps:
  - Step 1: Multiply the Weights by the inputs
  - Step 2: add the biases
  - step 3: apply an activation function of your choice (ReLU, tanh, sigmoid, etc.)
These will give you the activated values for each of the next nodes.

### Iteration 1: The First Layer of Weights

Let's use the neural network we have described from the previous layer:


#### Step 1
We multiply our previous layer's input by the corresponding row gives us the next node. When we do this, we are getting some feature representation of the previous layer and using that arrows as a means to put them into the node. For example, for node one, we would have ($w_{1,1}^{(1)} \text{  } w_{1,2}^{(1)}$), which represents all the weights (white arrows) going to a specific node. Multiplying this by the inputs $(x_1 \text{  } x_2)$ we can compute $w_1^{(1)}x$ to get the value for node one before being activated:


**For node 1 in layer one (the top node):**

$$
\left( \begin{array}{cccc}
w_{1,1}^{(1)} & w_{1,2}^{(1)}
\end{array} \right) 
\left( \begin{array}{cccc}
x_1  \\[0.5em]
x_2
\end{array} \right)
=
w_{1,1}^{(1)}x_1 + w_{1,2}^{(1)}x_2 = w_1^{(1)}x
$$

We can do the math for the entire layer by representing the multiplications in matrix form, with each subsequent node represented by its row position in the weight matrix:
$$
\left( \begin{array}{cccc}
w_{1,1}^{(1)} & w_{1,2}^{(1)} \\[0.5em]
w_{2,1}^{(1)} & w_{2,2}^{(1)} \\[0.5em]
w_{3,1}^{(1)} & w_{3,2}^{(1)} \\[0.5em]
w_{4,1}^{(1)} & w_{4,2}^{(1)} \\[0.5em]
\end{array} \right) 
\left( \begin{array}{cccc}
x_1  \\[0.5em]
x_2
\end{array} \right) = 
\left( \begin{array}{cccc}
w_{1,1}^{(1)}x_1 + w_{1,2}^{(1)}x_2 \\[0.5em]
w_{2,1}^{(1)}x_1 + w_{2,2}^{(1)}x_2 \\[0.5em]
w_{3,1}^{(1)}x_1 + w_{3,2}^{(1)}x_2 \\[0.5em]
w_{4,1}^{(1)}x_1 + w_{4,2}^{(1)}x_2 \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(1)}x \\[0.5em]
w_2^{(1)}x \\[0.5em]
w_3^{(1)}x \\[0.5em]
w_4^{(1)}x \\[0.5em]
\end{array} \right) 
$$

#### Step 2
Let's ADD bias 1 to node 1 as such, for example:

$$w_1^{(1)}x + b_1^{(1)}$$

**It is important that you do not multiply the bias. Only add. This is a common mistake many people make.** As you can see, adding bias correlates to the **red arrows** coming from the diagram's input bias into the next layer. We can represent adding bias to all nodes in matrix notation: 

$$
\left( \begin{array}{cccc}
w_1^{(1)}x \\[0.5em]
w_2^{(1)}x \\[0.5em]
w_3^{(1)}x \\[0.5em]
w_4^{(1)}x \\[0.5em]
\end{array} \right) 
+ \left( \begin{array}{cccc}
b_{1}^{(1)} \\[0.5em]
b_{2}^{(1)} \\[0.5em]
b_{3}^{(1)} \\[0.5em]
b_{4}^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(1)}x + b_{1}^{(1)} \\[0.5em]
w_2^{(1)}x + b_{2}^{(1)} \\[0.5em]
w_3^{(1)}x + b_{3}^{(1)}\\[0.5em]
w_4^{(1)}x + b_{4}^{(1)}\\[0.5em]
\end{array} \right) 
$$
We will denote this matrix as $z^{(1)}$:
$$
z^{(1)} = 
\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
z_{3}^{(1)} \\[0.5em]
z_{4}^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(1)}x + b_{1}^{(1)} \\[0.5em]
w_2^{(1)}x + b_{2}^{(1)} \\[0.5em]
w_3^{(1)}x + b_{3}^{(1)}\\[0.5em]
w_4^{(1)}x + b_{4}^{(1)}\\[0.5em]
\end{array} \right) 
$$

We are now at this location of the graph, without activation. We need to activate it before we send our "nodes" to the next layer.

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

#### Step 3
Finally, we can apply an activation, that we mentioned a previously. For the sake of ease, we let $f(x)$ be any non-linear [activation function](####Types-of-activation-functions). Then we have:
$$
f(z^{(1)}) = 
f\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
z_{3}^{(1)} \\[0.5em]
z_{4}^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} \\[0.5em]
\text{Node 2}^{(1)} \\[0.5em]
\text{Node 3}^{(1)} \\[0.5em]
\text{Node 4}^{(1)} \\[0.5em]
\end{array} \right)=

\left( \begin{array}{cccc}
a_1^{(1)} \\[0.5em]
a_2^{(1)} \\[0.5em]
a_3^{(1)} \\[0.5em]
a_4^{(1)} \\[0.5em]
\end{array} \right) = a^{(1)}
$$

where $a$ stands for activated.


You would repeat this process over and over again until you reached the end of the network. Let's do the next layer, just to see what this would look like.
### Iteration 2: The Second Layer of Weights
If we followed the equations correctly, you will now have the values for the first hidden layer. Let's focus on this layer, as denoted by the blue arrow:


In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    bias=True
)

# Display the plot.
nn_plot.show()

#### Step 1
We multiply our nodes by the new weight matrices. Notice how this matrix multiplication forces the numbers to be in the same dimensions as the next node layer. A plus!

$$
\left( \begin{array}{cccc}
w_{1,1}^{(2)} & w_{1,2}^{(2)} & w_{1,3}^{(2)} & w_{1,4}^{(2)}\\[0.5em]
w_{2,1}^{(2)} & w_{2,2}^{(2)} & w_{2,3}^{(2)} & w_{2,4}^{(2)} \\[0.5em]
\end{array} \right) 
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} \\[0.5em]
\text{Node 2}^{(1)} \\[0.5em]
\text{Node 3}^{(1)} \\[0.5em]
\text{Node 4}^{(1)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
w_{1,1}^{(2)}a_1^{(1)} + w_{1,2}^{(2)}a_2^{(1)} + w_{1,3}^{(2)}a_3^{(1)} + w_{1,4}^{(2)}a_4^{(1)} \\[0.5em]
w_{2,1}^{(2)}a_1^{(1)} + w_{2,2}^{(2)}a_2^{(1)} + w_{2,3}^{(2)}a_3^{(1)} + w_{2,4}^{(2)}a_4^{(1)}\\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(2)}a^{(1)} \\[0.5em]
w_2^{(2)}a^{(1)} \\[0.5em]
\end{array} \right) 
$$
**Notice the pattern here. Every row represents all of the "arrows" going into that node, and every "arrow" multiplies the previous layer's input, $a$, but some weight. When we perform the multiplication, all the the numbers correctly format to the number of nodes in the next layer!**
Also, notice that any node $i$ in layer $\ell$ is the same thing as $a_i^{(\ell)}$.

#### Step 2
We now add our bias, as deonted by the red arrows on the diagram.
$$
\left( \begin{array}{cccc}
w_1^{(2)}a^{(1)} \\[0.5em]
w_2^{(2)}a^{(1)} \\[0.5em]
\end{array} \right) 
+ \left( \begin{array}{cccc}
b_{1}^{(2)} \\[0.5em]
b_{2}^{(2)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
w_1^{(2)}a_1^{(1)} + b_{1}^{(2)} \\[0.5em]
w_2^{(2)}a_1^{(1)} + b_{2}^{(2)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
z_1^{(2)} \\[0.5em]
z_2^{(2)} \\[0.5em]
\end{array} \right)  = z^{(2)}
$$

This puts us at this current location of the graph:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

#### Step 3
We can see that this is actually the last weight layer, and given that this is a classification head, we have a softmax activation. This means that
$$f(x) = \frac{e^{z_i}}{e^{z_1}+e^{z_2}}, \text{ where i is whatever node we are coming from.} $$

We can apply it as such:
$$
f(z^{(2)}) =
f\left( \begin{array}{cccc}
z_1^{(2)} \\[0.5em]
z_2^{(2)} \\[0.5em]
\end{array} \right)  = 
\Large\left( \begin{array}{cccc}
\frac{e^{z_1^{(2)}}}{e^{z_1^{(2)}} + e^{z_2^{(2)}}} \\[0.5cm]
\frac{e^{z_2^{(2)}}}{e^{z_1^{(2)}} + e^{z_2^{(2)}}}  \\[0.5em]
\end{array} \right) =
\normalsize\left( \begin{array}{cccc}
a_1^{(2)} \\[0.5em]
a_2^{(2)}  \\[0.5em]
\end{array} \right) = 
a^{(2)} 
$$

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 2, 2],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=3,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

We have now finished forward pass!

### Alternative Forward pass: What if we were doing a regression problem?

If we were doing regression, our neural network will look more like this, as we are predicting some numerical value.

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 1, 1],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    bias=True
)

# Display the plot.
nn_plot.show()

### Iteration 1: 
This would be exactly the same as the previous problem:
#### Step 1:
$$
\left( \begin{array}{cccc}
w_{1,1}^{(1)} & w_{1,2}^{(1)} \\[0.5em]
w_{2,1}^{(1)} & w_{2,2}^{(1)} \\[0.5em]
w_{3,1}^{(1)} & w_{3,2}^{(1)} \\[0.5em]
w_{4,1}^{(1)} & w_{4,2}^{(1)} \\[0.5em]
\end{array} \right) 
\left( \begin{array}{cccc}
x_1  \\[0.5em]
x_2
\end{array} \right) = 
\left( \begin{array}{cccc}
w_{1,1}^{(1)}x_1 + w_{1,2}^{(1)}x_2 \\[0.5em]
w_{2,1}^{(1)}x_1 + w_{2,2}^{(1)}x_2 \\[0.5em]
w_{3,1}^{(1)}x_1 + w_{3,2}^{(1)}x_2 \\[0.5em]
w_{4,1}^{(1)}x_1 + w_{4,2}^{(1)}x_2 \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(1)}x \\[0.5em]
w_2^{(1)}x \\[0.5em]
w_3^{(1)}x \\[0.5em]
w_4^{(1)}x \\[0.5em]
\end{array} \right) 
$$
**Note:** $W^{(1)}$ has dimensions $4 \text{ x } 2$ because the next layer has 4 nodes and the previous layer had 2 nodes/features.

#### Step 2:
$$
\left( \begin{array}{cccc}
w_1^{(1)}x \\[0.5em]
w_2^{(1)}x \\[0.5em]
w_3^{(1)}x \\[0.5em]
w_4^{(1)}x \\[0.5em]
\end{array} \right) 
+ \left( \begin{array}{cccc}
b_{1}^{(1)} \\[0.5em]
b_{2}^{(1)} \\[0.5em]
b_{3}^{(1)} \\[0.5em]
b_{4}^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(1)}x + b_{1}^{(1)} \\[0.5em]
w_2^{(1)}x + b_{2}^{(1)} \\[0.5em]
w_3^{(1)}x + b_{3}^{(1)}\\[0.5em]
w_4^{(1)}x + b_{4}^{(1)}\\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
z_{3}^{(1)} \\[0.5em]
z_{4}^{(1)} \\[0.5em]
\end{array} \right) 
 =z^{(1)}
$$

We are now in this portion of our network:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 1, 1],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

#### Step 3:
We now activate our nodes, just like we did previously.
$$
f(z^{(1)}) = 
f\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
z_{3}^{(1)} \\[0.5em]
z_{4}^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} \\[0.5em]
\text{Node 2}^{(1)} \\[0.5em]
\text{Node 3}^{(1)} \\[0.5em]
\text{Node 4}^{(1)} \\[0.5em]
\end{array} \right)=

\left( \begin{array}{cccc}
a_1^{(1)} \\[0.5em]
a_2^{(1)} \\[0.5em]
a_3^{(1)} \\[0.5em]
a_4^{(1)} \\[0.5em]
\end{array} \right) = a^{(1)}
$$

We are now here in our neural network:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 1, 1],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="layer",
    bias=True
)

# Display the plot.
nn_plot.show()

### Iteration 2:

This is where things get to be a little different. Since we are only predicting one numerical value, we only have one output node. 

  - **Note:** It is possible to have multiple nodes for regression. What this would mean is we are predicting multiple values at onces. For example, if we have house size, lot size, and the numnber of bathroom as our input (x1,x2,x3), we  could have 2 regression heads:
    - One predicts house price (y1)
    - One predicts the number of bedrooms (y2)
  - This effectively predicting 2 values at once.

#### Step 1:
Multiply our weights by the nodes:

$$
\left( \begin{array}{cccc}
w_{1,1}^{(2)} & w_{1,2}^{(2)} & w_{1,3}^{(2)} & w_{1,4}^{(2)}\\[0.5em]
\end{array} \right) 
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} \\[0.5em]
\text{Node 2}^{(1)} \\[0.5em]
\text{Node 3}^{(1)} \\[0.5em]
\text{Node 4}^{(1)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
w_{1,1}^{(2)}a_1^{(1)} + w_{1,2}^{(2)}a_2^{(1)} + w_{1,3}^{(2)}a_3^{(1)} + w_{1,4}^{(2)}a_4^{(1)} \\[0.5em]
\end{array} \right) =
\left( \begin{array}{cccc}
w_1^{(2)}a^{(1)} \\[0.5em]
\end{array} \right) 
$$


Notice how we only have 1 node in the next layer, our node can be represented in a 1x1 matrix.

#### Step 2:
Add our bias:

$$
\left( \begin{array}{cccc}
w_1^{(2)}a^{(1)} \\[0.5em]
\end{array} \right) 
+ \left( \begin{array}{cccc}
b_{1}^{(2)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
w_1^{(2)}a_1^{(1)} + b_{1}^{(2)} \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
z_1^{(2)} \\[0.5em]
\end{array} \right)  = z^{(2)}
$$

We are now in the following portion of out network:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 1, 1],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

#### Step 3: 

We finish our forward pass using the regression activation. This means the layer is just linear, so we dont need to apply anything, we already have a numeric number! This means that the activation for this layer is just
$$
f(z^{(2)}) =
f\left( \begin{array}{cccc}
z^{(2)} \\[0.5em]
\end{array} \right)  =
\left( \begin{array}{cccc}
z^{(2)} \\[0.5em]
\end{array} \right) =
\normalsize\left( \begin{array}{cccc}
a^{(2)} \\[0.5em]
\end{array} \right) = 
a^{(2)} 
$$
This puts us here:


In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 4, 1, 1],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=3,
    special_inter_layer_arrow_offset="node",
    bias=True
)

# Display the plot.
nn_plot.show()

Since we are at our output layer, this is actually just the output itself. This concludes forward pass through the entire network. If you wish to have more practice, I recommend asking ChatGPT for small neural networks to feed forward through. Then ask if you get it right.

### Final Forward Pass Algorithm
In conclusion, each forward pass for the nodes in layer l can be can be summed up to the following Equation:

$$\Large a^{(\ell)} = f(W^{(\ell)}\cdot a^{(\ell-1)} + b^{\ell})$$

where
  - $f(x)$ is the [activation function](##Introduction-to-Neural-Networks)
  - $a^{(\ell)}$ is a layer's output from its nodes
  - $a^{(\ell-1)}$ is the previous layer nodes, which I have shown to be the matrix $(\text{Node 1}^{(1)}, \text{Node 2}^{(1)},\cdots)$ when we were computing the final output, for example (Iteration 2)
  - $W^{(\ell)}$ is the weight matrix for that layer
  - $b^{\ell}$ is the bias for that layer
  - $\cdot$ is a matrix multiplication
  - $a^{(1)}$ is always one sample in your dataset (a single row of the dataset). It has d "nodes" (number of columns (features), without the classification labels)
  - The following are equivalent: $z^{(\ell)} = W^{(\ell)}\cdot a^{(\ell-1)} + b^{\ell}$ and $a^{(\ell)} = f(z^{(\ell)})$
  - $z^{(\ell)}$ is the node before being activated by its respective activation function


Using this, you should effectively be able to calculate the "predictions" a neural network makes, depending on the input feature given for **one sample in a dataset**.

## Making our Models Learn: Backpropagation

---

We have covered how to predict a number with our neural network. However, if our model doesn't learn, how will it ever predict anything correctly? This is why we have to slowly "tune" our model so that it's weights will correctly predict what we want it to. We do this by using gradients to move in the direction that reduces this loss as much as possible.

### What are Gradients?

If you have every taken a calculus class before, you know what a derivative is: the change in a function at any given point f(x). The difference between derivatives and gradients lies in the fact that a gradient can be taken with respect to multiple variables, with their partial derivatives. This gradient is a **vector** of derivatives that not only tell you the magnitude of change, but also the **direction** of greatest increase. This is really useful, because most problems in machine learning are never just a single variable. Let's see an example:

Assume we have the function
$$f(x,y) = x^2+y^2$$

In [ ]:
plot_obj = PlotlySurfaceWithGradient(title="f(x,y)=x^2+y^2")
plot_obj()

Let's say we take the gradient at point (0.2,0.2):


In [ ]:
plot_obj = PlotlySurfaceWithGradient(title="Gradient at point (.2,.2)")
plot_obj((-.2,.2))

and the gradient at (.5,.5):

In [ ]:
plot_obj = PlotlySurfaceWithGradient(title="Gradient at point (.5,.5)")
plot_obj((-.5,.5))

Notice how much bigger the arrow is. If we flip this around this can correspond to how "steep" the function is.



In [ ]:
plot_obj = PlotlySurfaceWithGradient(title="Negative Gradient at point (.5,.5)",flip=True)
plot_obj((-.5,.5))

 But how does this relate to training our model?
 
### Minimizing Loss with Gradients: Gradient Descent

We had shown earlier the different loss fucntions we use for a neural network. If we were to review them again, we see that that we have the following loss functions:

**Classification:**
$$\text{ categorical cross entropy loss }= -\frac{1}{n}\sum_{i=1}^n \hat{y_i}\log(p_i)\text{. }$$
**Regression:**
$$MSE \text{ loss }= -\frac{1}{n}\sum_{i=1}^n (\hat{y_i}-\bar{y_i})^2\text{. }$$ 
  - **Reminder:** These aren't the only loss functions, just the most common.

What this means is that the loss might be like some function like we see above, where the smallest loss is at the lowest point (minima). However, since every single sample in our dataset will generate a different function, we dont ever really know what the loss functions look like or even is for that matter. We just randomly "spawn" on some part of this function, and try to guess where we should go. This means we don't know what our "true" loss function looks like. We are blind, which means we want to take a "step" in the direction in which we decrease loss. Since the point with the smallest loss means we get the best prediction, the intuition is that **we can change the weights with respect to the change in loss in the negative direction to take a step towards this minima,** until we reach the minima. This is the idea behind backpropagation for gradient descent.

If we take the gradient of the loss functions, we get the following values:

**Classification:**
$$softmax' \text{ for a single sample } = (p_i-\hat{y_i})\text{. }$$
**Regression:**
$$MSE' \text{ for a single sample }= \hat{y_i}-\bar{y_i}\text{. }$$


the following is a key for all of the variables:
  - $p_i$ is the softmax function
  - $\hat{y_i}$ is the true value that we are trying to predict for the label $i$ in our dataset.
  - $\bar{y_i}$ is the value we predicted with our model
  - $x_i$ are the values we get from the last layer.
  - $\frac{1}{n}\sum_{i=1}^n$ means the "mean" of the values.

If you look closely, the gradients of the loss quantify how "far" our prediction was from the actual values. By changing all the weights in our matrix, subtracting the gradients of the weights with respect to the loss for every single weight, we do a single step of backpropagation. If we do this backpropagation step until we reach the loss minimum, we will successfully perform gradient descent, our model will eventually "converge" to the local minima, and be able to predict the values we want!

### Backpropagation: The Math Behind Each Backwards Step

While in theory backpropagation might make sense, this is generally one of the hardest concepts to grasp the neural network. This is because backpropagation compounds on the fact that in order to calulate the gradient of the weights, (which is a weight matrix, as we had shown in forward pass), we have to chain the derivatives of the loss with the activations and pre-activations to be able to get the corresponding weight gradient. This means we have to calculate the gradient of the weights and biases for every layer, and tune them individually. For example, let's look at the following example:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    bias=True
)

nn_plot.show()

As a disclaimer, I have no idea how this was derived. I only know the formulas. If we want to calculate the gradients of all of the weights with respect to the loss for our example network, it would look something like the following:

$$\nabla_{W^{(3)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial W^{(3)}}$$
$$\nabla_{b^{(3)}} = \frac{\partial L}{\partial z^{(3)}}$$
$$\nabla_{W^{(2)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial W^{(2)}}$$
$$\nabla_{b^{(2)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}$$
$$\nabla_{W^{(1)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial a^{(1)}}\cdot\frac{\partial a^{(1)}}{\partial z^{(1)}}\cdot\frac{\partial z^{(1)}}{\partial W^{(1)}}$$
$$\nabla_{b^{(1)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial a^{(1)}}\cdot\frac{\partial a^{(1)}}{\partial z^{(1)}}$$
This essentially means you are calulating the loss of the current layer by looking at the loss from the next layer and multiplying it by the derivatives of the previous layer's activated values and unactivated values.

**Tip:** If you are coding this yourself, it's a good idea to save the pre-activated ($z$) and activated ($a$) values seperately. 

Where
  - $\large\frac{\partial L}{\partial z^{(i)}}$ is the direction of change needed to adjust the third layer's pre-activated z for the propagation of the loss
  - $\large\frac{\partial z^{(i)}}{\partial a^{(i-1)}}$  is the change needed to adjust the previous layer's node (activated value) for the propogation of the loss
  - $\large\frac{\partial a^{(i)}}{\partial z^{(i)}}$ is the change needed to adjust the current layer's value based on the activation function. Since the activation function is a function itself, that's how the chain rule ends up growing in length the more you go through layers.


It's a little hard to visualize these derivative changes, so let's try and work through the math. I know this is pretty complex, but notice this pattern here:
  - We only need to compute the derivative with respect to the weight we care about
  - Otherwise, its just some combination of $a$ (the node output for that layer), and $z$ (the preactivated node output for that layer-think before we apply our activation function) propagated from layers previous. This means that once we compute all of the other previous values, we just have to compute the next one in succession!
  - You also already compute the gradient for the bias of each layer before finding the gradient of the weights

Knowing all of this, **it's possible to set up a recursive formula to simplify it, like you would a loop in a coding program!** This part of the program will be pretty math-intensive, so feel free to go back and reread it. I will also have examples of me using backpropagation in examples, so you can get a grasp of it. IF you want even more practice, I personally learned backpropagation by having ChatGPT generate me small feed forward and backpropagation problems for both categorical and regression tasks that I could do by hand, with about 1-2 hidden layers each. Practice makes perfect.


#### What the first $\delta$ is:
For our purposes, $\delta$ just means calculated loss, and likewise $\delta^{(\ell)}$ is the loss for that specific layer. The final layer, $\delta^{(\ell)}$ is just our derivative loss values, or $\large\frac{\partial L}{\partial z^{(3)}}$. Do you remember the $(p_i-\hat{y_i})$ from above? For a hypothetical example, we have 3 categorical variables. Let's say we're trying to predict a number being either 1, 2, or 3, just to give meaning to the classification head, and let's say for the particular example we give it the true value is a 2. This means our $\hat{y_i}=$
$$
\left( \begin{array}{cccc}
0\\[0.5em]
1\\[0.5em]
0\\[0.5em]
\end{array}\right)
$$
then let's assume our model predicts the following values for each category with the softmax:

$$
\left( \begin{array}{cccc}
.39\\[0.5em]
.33\\[0.5em]
.28\\[0.5em]
\end{array}\right)
$$

then the loss with respect to the last layer for our hypthetical values will be

$$
\delta^{(\ell)} = 
\left( \begin{array}{cccc}
.39\\[0.5em]
.33\\[0.5em]
.28\\[0.5em]
\end{array}\right) -
\left( \begin{array}{cccc}
0\\[0.5em]
1\\[0.5em]
0\\[0.5em]
\end{array}\right)  
 = 
\left( \begin{array}{cccc}
-.61\\[0.5em]
.67\\[0.5em]
-.28\\[0.5em]
\end{array}\right)
$$
You can think of it like we are trying to lower the values that are wrong and increase the values that are right! I will give an example of regression for this at the bottom of the document.


#### Deriving the update rule

The most important thing to deriving the update rule is noticing patterns. We can see that any loss for the next layer is simply the loss for the previous layer, with the following values tacked on: $$\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}$$
Then that means we can let $\delta^{(\ell)}$ be the loss of the function per layer. If we look at the pattern I showed you we can build a recursive rule to build a loop with: $\large\delta^{(\ell)}\cdot\frac{\partial z^{(\ell)}}{\partial a^{(\ell-1)}}\cdot\frac{\partial a^{(\ell-1)}}{\partial z^{(\ell-1)}}$.


To calculate the loss $\delta^{(\ell-1)}$. We know that $z^{(\ell)} = W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}$, from our forward pass, before activation. We can start from here, because we already computed the derivative of the final layer doing ($p_i-y_i$). This means that taking the derivative of this function via the chain rule would therefore look something like this:
$$\frac{\partial z^{(\ell)}}{\partial a^{(\ell-1)}}[z^{(\ell)} = W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}] = W^{(\ell)}$$
since the bias term just goes away. However for our second derivative $\large\frac{\partial a^{(\ell-1)}}{\partial z^{(\ell-1)}}$, we use the function $a^{(\ell-1)} = f(W^{(\ell-1)}\cdot a^{(\ell-2)} + b^{(\ell-1)})$. Notice how this is functionally equivalent to $a^{(\ell-1)} = f(z^{(\ell-1)})$. Then
$$\frac{\partial a^{(\ell-1)}}{\partial z^{(\ell-1)}}[f(z^{(\ell-1)})] = f'(z^{(\ell-1)})\cdot 1 = f'(z^{(\ell-1)})$$
where $f'(x)$ is the derivative of the activation function for the layer $\ell-1$.

Notice that this z vector is a vector, not a matrix corresponding to the size of the nodes. This means when we finally construct our update rule, we need to do element-wise multiplication, via the hadamard product. 

#### Mini-lesson on Hadamard Product
You can only multiply these 2 matrices/vectors if they are the same dimensions, and every element in one matrix A $a_{i\text{ , } j}$ will be multiplied by its corresponding coordinates in matrix B $b_{i\text{ , } j}$ to get a matrix AB = $a_{i\text{ , } j}*b_{i\text{ , } j}$. For example, let's say I have 2 matrices

$$
A = \left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right) \text{ and }
B = \left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right)
$$

Their hadamard product (element-wise matrix multiplication) would be 

$$
\left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right)\odot
\left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right) = 
\left( \begin{array}{cccc}
1 & 4 & 9\\[0.5em]
16 & 25 & 36\\[0.5em]
49 & 64 & 81\\[0.5em]
\end{array}\right) = AB
$$

normal matrix multiplication on the other hand would yield
$$
\left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right) \text{X}
\left( \begin{array}{cccc}
1 & 2 & 3\\[0.5em]
4 & 5 & 6\\[0.5em]
7 & 8 & 9\\[0.5em]
\end{array}\right) = 
\left( \begin{array}{cccc}
30 & 36 & 42\\[0.5em]
66 & 81 & 96\\[0.5em]
102 & 126 & 150\\[0.5em]
\end{array}\right) = AB
$$

#### Back to Propagation
with this now in mind, we put our derivations together to get our new update rule for each layer's loss:

$$\large\delta^{(\ell-1)} = \delta^{(\ell)}\cdot\frac{\partial z^{(\ell)}}{\partial a^{(\ell-1)}}\cdot\frac{\partial a^{(\ell-1)}}{\partial z^{(\ell-1)}} = W^{(\ell)T}\delta^{(\ell)} \odot f'(z^{(\ell-1)})
$$

or less confusingly, 
$$\Large\delta^{(\ell-1)} = W^{(\ell)T}\delta^{(\ell)} \odot f'(z^{(\ell-1)}).$$
This is the loss for every layer! As a reminder:
  - $(\ell)$ is the layer
  - $\large \delta^{(\ell-1)}$ is the loss for the layer $\ell-1$ 
  - $\large W^{(\ell)}$ is the weight matrix for layer $\ell$
  - $\large \delta^{(\ell)}$ is the loss for the layer $\ell$
  - $\large z^{(\ell-1)}$ is the values calculated for the node BEFORE it is sent through the activation function for that layer
  - $\large f(z^{(\ell)})$ is the activation function for the layer corresponding to the layer of it's z value
  - $\large f'(x)$ is the derivative of the activation function
  - $\large f'(z^{(\ell-1)})$ are the preactivated nodes from the $\ell-1$ layer, **put through the DERIVATIVE of their activation.** <- This is common source of confusion.
  - $\odot$ is the hadamard product, aka element-wise multiplication (**DIFFERENT FROM MATRIX MULTIPLICATION**)
  - $^T$ is transpose, where we switch the rows to be the columns of a matrix, and the columns to be the rows.

#### Getting the Gradient of the bias and Weight Matrices
The most important part about making update rules is looking for patterns. Look at the following: What do you notice?

##### Bias
Let's look at our previous examples for bias derivations:
$$\nabla_{b^{(3)}} = \frac{\partial L}{\partial z^{(3)}}$$
$$\nabla_{b^{(2)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}$$
$$\nabla_{b^{(1)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial a^{(1)}}\cdot\frac{\partial a^{(1)}}{\partial z^{(1)}}$$

Did you notice that this is exactly the loss of every single layer? This means that for our update rule, once we find our the loss we find our bias! in other words
$$\Large\delta^{(\ell)}=\nabla_{b^{(\ell)}}$$

##### Weights
for $\large\nabla_{W^{(\ell)}}$, we notice that the only difference is now we tack on $\large\frac{\partial z^{(3)}}{\partial W^{(3)}}$ or 
$$\nabla_{W^{(3)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(\ell)}}{\partial W^{(\ell)}}$$
$$\nabla_{W^{(2)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial W^{(2)}}$$
$$\nabla_{W^{(1)}} = \frac{\partial L}{\partial z^{(3)}}\cdot\frac{\partial z^{(3)}}{\partial a^{(2)}}\cdot\frac{\partial a^{(2)}}{\partial z^{(2)}}\cdot\frac{\partial z^{(2)}}{\partial a^{(1)}}\cdot\frac{\partial a^{(1)}}{\partial z^{(1)}}\cdot\frac{\partial z^{(1)}}{\partial W^{(1)}}$$

all of these are essentially loss multiplied by the derivative of the weight matrix (gradient). So then we can build the following rule, by looking at the patterns:
$$\large\nabla_{W^{(\ell)}} = \delta^{(\ell)}\cdot\frac{\partial z^{(\ell)}}{\partial W^{(\ell)}}$$
if we deconstruct this derivative, the top is the function $z^{(\ell)}$. We then can grab the function corresponding: $z^{(\ell)} = W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}$. Then 
$$\frac{\partial z^{(\ell)}}{\partial W^{(\ell)}}[W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}] =  a^{(\ell-1)}$$
Notice that $\delta{(\ell)}$ and $a^{(\ell-1)}$ do not match dimension, nor will they create a matrix of the same dimensions of the weight matrix. They are both dimensions $\large n^{(\ell)}\text{x}1$ and $\large n^{(\ell-1)}\text{x}1$ respectively, where $n$ is the number of nodes in the layer. IF we want to make it the size of the weight matrix, we do $\large a^{(\ell-1)T}$ so we get dimensions $\large n^{(\ell)}\text{x}1$ and $\large 1\text{x}n^{(\ell-1)}$ for the matrix outer product, which matches the dimensions of the weight matrix. We can now find out what the update rule for the gradient of a weight matrix of any given layer!
$$\Large\nabla_{W^{(\ell)}}=\delta^{(\ell)}\cdot a^{(\ell-1)T}$$

### Final Update Rules
Based on the following update rules, we have deduced recursively the update rules for any given layer, and they are as follows:

$$\Large\text{The first }\delta^{(\ell)} = (p_i-\hat{y_i}) \text{ for classification, or } (\hat{y_i}-\bar{y_i}) \text{ for regression.}$$
$$\Large\nabla_{W^{(\ell)}}=\delta^{(\ell)} a^{(\ell-1)T}$$
$$\Large\nabla_{b^{(\ell)}} = \delta^{(\ell)}$$
$$\Large\delta^{(\ell-1)} = W^{(\ell)T}\delta^{(\ell)} \odot f'(z^{(\ell-1)}).$$

Where:
  - $\hat{y_i}$ is the predicted value generated by your network's forward pass for any given category, for a certain sample.
  - $\bar{y_i}$ is the actual value/label given to you for any given category, for a certain sample. This will be 0/1 for classification and some real number for regression.
  - $p_i$ is the softmax function.
  - $(\ell)$ is the layer
  - $\large W^{(\ell)}$ is the weight matrix for layer $\ell$
  - $\large \delta^{(\ell)}$ is the loss for the layer $\ell$
  - Likewise, $\large \delta^{(\ell-1)}$ is the loss for the layer $\ell-1$ 
  - $\large z^{(\ell-1)}$ is the values calculated for the node BEFORE it is sent through the activation function for that layer
  - $\large f(z^{(\ell)})$ is the activation function for the layer corresponding to the layer of it's preactivated nodes
  - $\large f'(x)$ is the derivative of the activation function
  - $\large f'(z^{(\ell-1)})$ are the preactivated nodes from the $\ell-1$ layer, **put through the DERIVATIVE of their activation.** <- This is common source of confusion.
  - $\odot$ is the hadamard product, aka element-wise multiplication (**DIFFERENT FROM MATRIX MULTIPLICATION**)
  - $^T$ is transpose, where we switch the rows to be the columns of a matrix, and the columns to be the rows.
  - $\large\delta^{(\ell)}\cdot a^{(\ell-1)T}$ is an outer product
  - $\large a^{(\ell)} = f(W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}) = f(z^{(\ell)})$
  - $\large z^{(\ell)} = W^{(\ell)}\cdot a^{(\ell-1)} + b^{(\ell)}$

This is definitely a lot of information, so make sure to come back to this a couple of times to be able to get it.

### Weight updates (motivations of gradient descent)

Now that we are able to derive the gradients of the weights, we can take a step in the direction of these gradients. Naively, we can do this by subtracting the gradients from the current weights, and likewise with the biases:
$$W^{(\ell)}_{new} = W^{(\ell)} - \nabla W^{(\ell)}$$
$$b^{(\ell)}_{new} = b^{(\ell)} - \nabla b^{(\ell)}$$

This basic idea is how gradient descent works, however there is an important caveat: We might take too big of a step. Let's visualize how a parameter space for a loss function MIGHT look. Run the following simulation:

In [ ]:
sim = GradientDescentSimulator(learning_rate=50, title="Gradient Descent With no Learning Rate", max_iter=100)
sim.fig.show()

Do you see what happens? The gradient tells us to move quickly where it's steep, and as a result, we "overshoot" our minimum point of loss. We keep moving back and forth, and we will never reach the best minima.

Here's an analogy: Think you are holding a basketball in a dense fog, so dense that you can't see anything. This is basketball your neural network. Your goal is to throw the basketball into the hoop, but you obviously don't know where it is because you can't see You have some information telling you which direction to throw the basketball and it tells you how hard to throw the ball. Every time you throw the ball, you're automatically teleported to the ball, so you don't get to walk over the landscape to see if you can guess where to shoot. For some reason, the information telling you to throw it harder and harder when you're close to the goal. As a result, you're never able to get the ball into the hoop. You don't have all day!

We can see that in our simulation, the curve gets steeper the closer we get to the local minimum. As a result, the gradient tells us to throw our ball REALLY hard, because the negative gradient is steep, like we saw in the example above. So how do we fix this?

### Our first Hyperparameter: $\eta$ (learning rate) and the Gradient Descent Algorithm

We can add some small constant $\eta$ to the negative gradients to "dampen" their effects on the weights. This will effectively make us "toss" the basketball a bunch of times, so that we inch towards the goal. This kind of makes more sense, because we can't see! The classic gradient descent algorithm can therefore be written as such: 

$$\large W^{(\ell)}_{new} = W^{(\ell)} - \eta\nabla W^{(\ell)}$$
$$\large b^{(\ell)}_{new} = b^{(\ell)} - \eta\nabla b^{(\ell)}$$

where
 - $\eta$ is the learning rate. This is usually some small number, custom to every neural network problem! (You need to figure out the best one yourself)
 - $\nabla b^{(\ell)}$ is the gradient of the biases for that layer $\ell$
 - $\nabla W^{(\ell)}$ is the gradient of the Weight Matrix for that layer $\ell$
 - $W^{(\ell)}$ is the weight matrix for layer $\ell$
 - $b^{(\ell)}$ is the biases for layer $\ell$
 - $W^{(\ell)}_{new}$ is the updated weight matrix for layer $\ell$
 - $b^{(\ell)}_{new}$ is the updated biases matrix for layer $\ell$

Let's see what happens when we add this constant to our simulation:

In [ ]:
sim = GradientDescentSimulator(learning_rate=10, title="Gradient Descent With Good Learning Rate",max_iter=100)
sim.fig.show()


As you can see, we converge perfectly! Conversely it's important not to make the learning rate too low, or we have the opposite problem that we saw in the first simulation:

In [ ]:
sim = GradientDescentSimulator(learning_rate=1, title="Gradient Descent With Low Learning Rate",max_iter=100)
sim.fig.show()

 As you can see the ball moves so slowly that it will never reach its local minima in the time that we need it to. As a result we strain our computer, while not getting a good model.
 
 Now that we know what learning rate ($\eta$) is, we now have all the tools to go through a full backpropagation example.

### Backpropagation Example

 In the meantime, let's use our new update rules to apply backpropagation to our synthetic model we had defined previously:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=3,
    special_inter_layer_arrow_offset="node",
    bias=True
    
)

nn_plot.show()

### Layer 3

Let's recall our toolbox of update rules:
$$\text{The first }\delta^{(\ell)} = (p_i-\hat{y_i}) \text{ for classification, or } (\hat{y_i}-\bar{y_i}) \text{ for regression.}$$
$$\nabla_{W^{(\ell)}}=\delta^{(\ell)} a^{(\ell-1)T}$$
$$\nabla_{b^{(\ell)}} = \delta^{(\ell)}$$
$$\delta^{(\ell-1)} = W^{(\ell)T}\delta^{(\ell)} \odot f'(z^{(\ell-1)})$$

#### Step 1: Calculate initial loss
Because we have specified that this is a classification problem, we know:
$$\delta^{(3)} = 
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)$$

#### Step 2: Calculate the weight and bias gradients

We are now at this point on the graph:



In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=3,
    special_inter_layer_arrow_offset="layer",
    bias=True
    
)

nn_plot.show()

We can calculate our weights with the update rule $\nabla_{W^{(\ell)}}=\delta^{(\ell)} a^{(\ell-1)T}$.

$$\Large\nabla_{W^{(3)}} = \delta^{(3)} a^{(2)T}\normalsize = 
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)\cdot
\left( \begin{array}{cccc}
\text{Node 1}^{(2)} \\[0.5em]
\text{Node 2}^{(2)} \\[0.5em]
\text{Node 3}^{(2)}\\[0.5em]
\text{Node 4}^{(2)}\\[0.5em]
\end{array}\right)^T =
$$
where Node $i$ is the activated value coming out of that node, or $a_i^{(2)}$.
$$\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)\cdot
\left( \begin{array}{cccc}
\text{Node 1}^{(2)} & \text{Node 2}^{(2)}& \text{Node 3}^{(2)}& \text{Node 4}^{(2)}\\[0.5em]
\end{array}\right) =
$$
$$\left( \begin{array}{cccc}
(p_1 - y_1)\cdot\text{Node 1}^{(2)} & (p_1 - y_1)\cdot\text{Node 2}^{(2)} & (p_1 - y_1)\cdot\text{Node 3}^{(2)} & (p_1 - y_1)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_2 - y_2)\cdot\text{Node 1}^{(2)} & (p_2 - y_2)\cdot\text{Node 2}^{(2)} & (p_2 - y_2)\cdot\text{Node 3}^{(2)} & (p_2 - y_2)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_3 - y_3)\cdot\text{Node 1}^{(2)} & (p_3 - y_3)\cdot\text{Node 2}^{(2)} & (p_3 - y_3)\cdot\text{Node 3}^{(2)} & (p_3 - y_3)\cdot\text{Node 4}^{(2)}
\end{array}\right) 
$$

We calculate our bias to be via the update rule:

$$
\Large\nabla_{b^{(3)}} = \delta^{(3)} \normalsize= 
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)
$$

#### Step 3: Perform Gradient descent:
$$\large W^{(3)}_{new} = W^{(3)} - \eta\nabla W^{(3)}$$
$$\Large W^{(3)} \normalsize= \left( \begin{array}{cccc}
w_{1,1}^{(3)}& w_{1,2}^{(3)} & w_{1,3}^{(3)} & w_{1,4}^{(3)}\\[0.5em]
w_{2,1}^{(3)}& w_{2,2}^{(3)} & w_{2,3}^{(3)} & w_{2,4}^{(3)}\\[0.5em]
w_{3,1}^{(3)}& w_{3,2}^{(3)} & w_{3,3}^{(3)} & w_{3,4}^{(3)}\\[0.5em]
\end{array}\right) $$
$$\large\nabla_{W^{(3)}} \normalsize = 
\left( \begin{array}{cccc}
(p_1 - y_1)\cdot\text{Node 1}^{(2)} & (p_1 - y_1)\cdot\text{Node 2}^{(2)} & (p_1 - y_1)\cdot\text{Node 3}^{(2)} & (p_1 - y_1)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_2 - y_2)\cdot\text{Node 1}^{(2)} & (p_2 - y_2)\cdot\text{Node 2}^{(2)} & (p_2 - y_2)\cdot\text{Node 3}^{(2)} & (p_2 - y_2)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_3 - y_3)\cdot\text{Node 1}^{(2)} & (p_3 - y_3)\cdot\text{Node 2}^{(2)} & (p_3 - y_3)\cdot\text{Node 3}^{(2)} & (p_3 - y_3)\cdot\text{Node 4}^{(2)}
\end{array}\right)
$$

$$\large b^{(3)}_{new} = b^{(3)} - \eta\nabla b^{(3)}$$
$$\large b^{(3)} \normalsize = 
\left( \begin{array}{cccc}
b_{1}^{(3)} \\[0.5em]
b_{2}^{(3)} \\[0.5em]
b_{3}^{(3)} \\[0.5em]
\end{array}\right) 
$$
$$\large\nabla b^{(3)}=\delta^{(3)} \normalsize
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)$$

$$\large W^{(3)}_{new} = \normalsize\left( \begin{array}{cccc}
w_{1,1}^{(3)}& w_{1,2}^{(3)} & w_{1,3}^{(3)} & w_{1,4}^{(3)}\\[0.5em]
w_{2,1}^{(3)}& w_{2,2}^{(3)} & w_{2,3}^{(3)} & w_{2,4}^{(3)}\\[0.5em]
w_{3,1}^{(3)}& w_{3,2}^{(3)} & w_{3,3}^{(3)} & w_{3,4}^{(3)}\\[0.5em]
\end{array}\right) - \eta\left( \begin{array}{cccc}
(p_1 - y_1)\cdot\text{Node 1}^{(2)} & (p_1 - y_1)\cdot\text{Node 2}^{(2)} & (p_1 - y_1)\cdot\text{Node 3}^{(2)} & (p_1 - y_1)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_2 - y_2)\cdot\text{Node 1}^{(2)} & (p_2 - y_2)\cdot\text{Node 2}^{(2)} & (p_2 - y_2)\cdot\text{Node 3}^{(2)} & (p_2 - y_2)\cdot\text{Node 4}^{(2)} \\[0.5em]
(p_3 - y_3)\cdot\text{Node 1}^{(2)} & (p_3 - y_3)\cdot\text{Node 2}^{(2)} & (p_3 - y_3)\cdot\text{Node 3}^{(2)} & (p_3 - y_3)\cdot\text{Node 4}^{(2)}
\end{array}\right)$$

$$\large b^{(3)}_{new}\normalsize = 
\left( \begin{array}{cccc}
b_{1}^{(3)} \\[0.5em]
b_{2}^{(3)} \\[0.5em]
b_{3}^{(3)} \\[0.5em]
\end{array}\right) 
-\eta
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)$$


#### Layer 2: Calculate Loss for the New Layer
We are now here in our neural network:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="node",
    bias=True
    
)

nn_plot.show()

#### Step 1: Calculate loss
This part is a little tricky, but just remember our update rules. You can find activation functions and their derivatives above. Remember $a^{(\ell)}$ does NOT refer to the activation function, but rather the number we get from it. I use Node i and  $a_i^{(\ell)}$ interchangeably. Our update rule is as follows: $\delta^{(2)} = W^{(3)T}\delta^{(3)} \odot f'(z^{(2)})$

$$\large \delta^{(2)} \normalsize=  \left( \begin{array}{cccc}
w_{1,1}^{(3)}& w_{1,2}^{(3)} & w_{1,3}^{(3)} & w_{1,4}^{(3)}\\[0.5em]
w_{2,1}^{(3)}& w_{2,2}^{(3)} & w_{2,3}^{(3)} & w_{2,4}^{(3)}\\[0.5em]
w_{3,1}^{(3)}& w_{3,2}^{(3)} & w_{3,3}^{(3)} & w_{3,4}^{(3)}\\[0.5em]
\end{array}\right)^T
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)\odot
f'\left( \begin{array}{cccc}
z_{1}^{(2)} \\[0.5em]
z_{2}^{(2)} \\[0.5em]
z_{3}^{(2)} \\[0.5em]
z_{4}^{(2)} \\[0.5em]
\end{array}\right)
$$

$$\large \delta^{(2)} =  \left( \begin{array}{ccc}
w_{1,1}^{(3)} & w_{2,1}^{(3)} & w_{3,1}^{(3)} \\[0.5em]
w_{1,2}^{(3)} & w_{2,2}^{(3)} & w_{3,2}^{(3)} \\[0.5em]
w_{1,3}^{(3)} & w_{2,3}^{(3)} & w_{3,3}^{(3)} \\[0.5em]
w_{1,4}^{(3)} & w_{2,4}^{(3)} & w_{3,4}^{(3)}
\end{array}\right)
\left( \begin{array}{cccc}
p_1 - y_1 \\[0.5em]
p_2 - y_2 \\[0.5em]
p_3 - y_3\\[0.5em]
\end{array}\right)\odot
f'\left( \begin{array}{cccc}
z_{1}^{(2)}\\[0.5em]
z_{2}^{(2)}\\[0.5em]
z_{3}^{(2)}\\[0.5em]
z_{4}^{(2)}\\[0.5em]
\end{array}\right)
$$

to simplify this section, we will simplify each matrix knowing their constituent parts. We know for our $\delta^{(3)}, p_i - y_i$ is just some positive or negative in $-1\leq0\leq1$. This number can be represented as such: $\delta_i^{(3)}$. So we let $\delta^{(3)}$ = 
$$
\left( \begin{array}{cccc}
\delta_1^{(3)}\\[0.5em]
\delta_2^{(3)} \\[0.5em]
\delta_3^{(3)}\\[0.5em]
\end{array}\right)
$$

then 
$$W^{(3)T}\delta^{(3)} = \left( \begin{array}{ccc}
w_{1,1}^{(3)} & w_{2,1}^{(3)} & w_{3,1}^{(3)} \\[0.5em]
w_{1,2}^{(3)} & w_{2,2}^{(3)} & w_{3,2}^{(3)} \\[0.5em]
w_{1,3}^{(3)} & w_{2,3}^{(3)} & w_{3,3}^{(3)} \\[0.5em]
w_{1,4}^{(3)} & w_{2,4}^{(3)} & w_{3,4}^{(3)}
\end{array}\right)\left( \begin{array}{cccc}
\delta_1^{(3)}\\[0.5em]
\delta_2^{(3)} \\[0.5em]
\delta_3^{(3)}\\[0.5em]
\end{array}\right)\odot
f'\left( \begin{array}{cccc}
z_{1}^{(2)} \\[0.5em]
z_{2}^{(2)} \\[0.5em]
z_{3}^{(2)} \\[0.5em]
z_{4}^{(2)} \\[0.5em]
\end{array}\right) = 
$$
$$
\left(\begin{array}{cccc}
 w_{1,1}^{(3)}\delta_1^{(3)} + w_{2,1}^{(3)}\delta_2^{(3)} + w_{3,1}^{(3)}\delta_3^{(3)} \\[0.5em]
w_{1,2}^{(3)}\delta_1^{(3)} + w_{2,2}^{(3)}\delta_2^{(3)} + w_{3,2}^{(3)}\delta_3^{(3)} \\[0.5em]
w_{1,3}^{(3)}\delta_1^{(3)} + w_{2,3}^{(3)}\delta_2^{(3)} + w_{3,3}^{(3)}\delta_3^{(3)} \\[0.5em]
w_{1,4}^{(3)}\delta_1^{(3)} + w_{2,4}^{(3)}\delta_2^{(3)} + w_{3,4}^{(3)}\delta_3^{(3)} \\[0.5em]
\end{array}\right)\odot
f'\left( \begin{array}{cccc}
z_{1}^{(2)} \\[0.5em]
z_{2}^{(2)} \\[0.5em]
z_{3}^{(2)} \\[0.5em]
z_{4}^{(2)} \\[0.5em]
\end{array}\right)
$$

Where f'(x) is the DERIVATIVE of some [activation function](#types-of-activation-functions). We then get that 
$$\Large \delta^{(2)} = \normalsize
\left(\begin{array}{cccc}
(w_{1,1}^{(3)}\delta_1^{(3)} + w_{2,1}^{(3)}\delta_2^{(3)} + w_{3,1}^{(3)}\delta_3^{(3)})*f'(z_{1}^{(2)})\\[0.5em]
(w_{1,2}^{(3)}\delta_1^{(3)} + w_{2,2}^{(3)}\delta_2^{(3)} + w_{3,2}^{(3)}\delta_3^{(3)})*f'(z_{2}^{(2)}) \\[0.5em]
(w_{1,3}^{(3)}\delta_1^{(3)} + w_{2,3}^{(3)}\delta_2^{(3)} + w_{3,3}^{(3)}\delta_3^{(3)})*f'(z_{3}^{(2)})\\[0.5em]
(w_{1,4}^{(3)}\delta_1^{(3)} + w_{2,4}^{(3)}\delta_2^{(3)} + w_{3,4}^{(3)}\delta_3^{(3)})*f'(z_{4}^{(2)}) \\[0.5em]
\end{array}\right)
$$



notice how each row is a single number. We can simplify this as
$$\Large \delta^{(2)}\normalsize =
\left(\begin{array}{cccc}
(w_{1,1}^{(3)}\delta_1^{(3)} + w_{2,1}^{(3)}\delta_2^{(3)} + w_{3,1}^{(3)}\delta_3^{(3)})*f'(z_{1}^{(2)})\\[0.5em]
(w_{1,2}^{(3)}\delta_1^{(3)} + w_{2,2}^{(3)}\delta_2^{(3)} + w_{3,2}^{(3)}\delta_3^{(3)})*f'(z_{2}^{(2)}) \\[0.5em]
(w_{1,3}^{(3)}\delta_1^{(3)} + w_{2,3}^{(3)}\delta_2^{(3)} + w_{3,3}^{(3)}\delta_3^{(3)})*f'(z_{3}^{(2)})\\[0.5em]
(w_{1,4}^{(3)}\delta_1^{(3)} + w_{2,4}^{(3)}\delta_2^{(3)} + w_{3,4}^{(3)}\delta_3^{(3)})*f'(z_{4}^{(2)}) \\[0.5em]
\end{array}\right)=
\left( \begin{array}{cccc}
\delta_1^{(2)}\\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)}\\[0.5em]
\delta_4^{(2)}\\[0.5em]
\end{array}\right)
$$
We will use this matrix for our further calculations to avoid confusion.

We are now here in our backpropagation:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="layer",
    bias=True
)

nn_plot.show()

We will now go a little faster, now that we've seen the process.
#### Step 2: Calculate $\nabla_{W^{(2)}}$ and $\nabla_{b^{(2)}}$
Our update rule for this layer is $\nabla_{W^{(2)}}=\delta^{(2)} a^{(1)T}$ and for bias $\nabla_{b^{(2)}}=\delta^{(2)}$. 


Our weight gradient
$$\Large\nabla_{W^{(2)}} = \delta^{(2)} a^{(1)T}\normalsize = 
\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right)\cdot
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} \\[0.5em]
\text{Node 2}^{(1)} \\[0.5em]
\end{array}\right)^T =
$$
$$\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right)\cdot
\left( \begin{array}{cccc}
\text{Node 1}^{(1)} & \text{Node 2}^{(1)}\\[0.5em]
\end{array}\right) =
$$
$$\left( \begin{array}{cc}
\delta_1^{(2)}\cdot\text{Node 1}^{(1)} & \delta_1^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_2^{(2)}\cdot\text{Node 1}^{(1)} & \delta_2^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_3^{(2)}\cdot\text{Node 1}^{(1)} & \delta_3^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_4^{(2)}\cdot\text{Node 1}^{(1)} & \delta_4^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\end{array}\right)
$$

Calculating Bias gradient:

$$\nabla_{b^{(2)}}=\delta^{(2)} =\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right) $$


#### Step 3: Update weights and biases for layer 2 with gradient descent

for weights:
$$\Large W^{(2)} \normalsize = \left( \begin{array}{cc}
w_{1,1}^{(2)} & w_{1,2}^{(2)} \\[0.5em]
w_{2,1}^{(2)} & w_{2,2}^{(2)} \\[0.5em]
w_{3,1}^{(2)} & w_{3,2}^{(2)} \\[0.5em]
w_{4,1}^{(2)} & w_{4,2}^{(2)} \\[0.5em]
\end{array}\right)
$$
$$\Large\nabla_{W^{(2)}} \normalsize = \left( \begin{array}{cc}
\delta_1^{(2)}\cdot\text{Node 1}^{(1)} & \delta_1^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_2^{(2)}\cdot\text{Node 1}^{(1)} & \delta_2^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_3^{(2)}\cdot\text{Node 1}^{(1)} & \delta_3^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_4^{(2)}\cdot\text{Node 1}^{(1)} & \delta_4^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\end{array}\right)
$$

for bias:
$$b^{(2)} = \left( \begin{array}{cccc}
b_1^{(2)} \\[0.5em]
b_2^{(2)} \\[0.5em]
b_3^{(2)} \\[0.5em]
b_4^{(2)} \\[0.5em]
\end{array}\right)$$
$$\nabla_{b^{(2)}}=\delta^{(2)} =\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right) $$

step:
$$
\Large W_{new}^{(2)} \normalsize =
\left( \begin{array}{cc}
w_{1,1}^{(2)} & w_{1,2}^{(2)} \\[0.5em]
w_{2,1}^{(2)} & w_{2,2}^{(2)} \\[0.5em]
w_{3,1}^{(2)} & w_{3,2}^{(2)} \\[0.5em]
w_{4,1}^{(2)} & w_{4,2}^{(2)} \\[0.5em]
\end{array}\right) - \eta\left( \begin{array}{cc}
\delta_1^{(2)}\cdot\text{Node 1}^{(1)} & \delta_1^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_2^{(2)}\cdot\text{Node 1}^{(1)} & \delta_2^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_3^{(2)}\cdot\text{Node 1}^{(1)} & \delta_3^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\delta_4^{(2)}\cdot\text{Node 1}^{(1)} & \delta_4^{(2)}\cdot\text{Node 2}^{(1)} \\[0.5em]
\end{array}\right)
$$

$$
\Large b_{new}^{(2)} \normalsize =
b^{(2)} = \left( \begin{array}{cccc}
b_1^{(2)} \\[0.5em]
b_2^{(2)} \\[0.5em]
b_3^{(2)} \\[0.5em]
b_4^{(2)} \\[0.5em]
\end{array}\right) - \eta
\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right)
$$

### Layer 1
We are now here in our diagram:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    special_inter_layer_arrow_offset="node",
    bias=True
)

nn_plot.show()

#### Step 1: Calculate loss for the new layer
Our update rule for this layer's loss is $\delta^{(1)} = W^{(2)T}\delta^{(2)} \odot f'(z^{(1)})$
$$
\Large \delta^{(1)} = \normalsize\left( \begin{array}{cc}
w_{1,1}^{(2)} & w_{1,2}^{(2)} \\[0.5em]
w_{2,1}^{(2)} & w_{2,2}^{(2)} \\[0.5em]
w_{3,1}^{(2)} & w_{3,2}^{(2)} \\[0.5em]
w_{4,1}^{(2)} & w_{4,2}^{(2)} \\[0.5em]
\end{array}\right)^T\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right) \odot
f'\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
\end{array}\right) = 
$$
$$
\left( \begin{array}{cccc}
w_{1,1}^{(2)} & w_{2,1}^{(2)} & w_{3,1}^{(2)} & w_{4,1}^{(2)} \\[0.5em]
w_{1,2}^{(2)} & w_{2,2}^{(2)} & w_{3,2}^{(2)} & w_{4,2}^{(2)} \\[0.5em]
\end{array} \right)\left( \begin{array}{cccc}
\delta_1^{(2)} \\[0.5em]
\delta_2^{(2)} \\[0.5em]
\delta_3^{(2)} \\[0.5em]
\delta_4^{(2)} \\[0.5em]
\end{array}\right) \odot
f'\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
\end{array}\right) = 
$$
$$
\left( \begin{array}{c}
w_{1,1}^{(2)}\cdot\delta_1^{(2)} + w_{2,1}^{(2)}\cdot\delta_2^{(2)} + w_{3,1}^{(2)}\cdot\delta_3^{(2)} + w_{4,1}^{(2)}\cdot\delta_4^{(2)} \\[0.5em]
w_{1,2}^{(2)}\cdot\delta_1^{(2)} + w_{2,2}^{(2)}\cdot\delta_2^{(2)} + w_{3,2}^{(2)}\cdot\delta_3^{(2)} + w_{4,2}^{(2)}\cdot\delta_4^{(2)} \\[0.5em]
\end{array} \right) \odot
f'\left( \begin{array}{cccc}
z_{1}^{(1)} \\[0.5em]
z_{2}^{(1)} \\[0.5em]
\end{array}\right) = 
$$
$$
\left( \begin{array}{c}
(w_{1,1}^{(2)}\cdot\delta_1^{(2)} + w_{2,1}^{(2)}\cdot\delta_2^{(2)} + w_{3,1}^{(2)}\cdot\delta_3^{(2)} + w_{4,1}^{(2)}\cdot\delta_4^{(2)})*f'(z_{1}^{(1)}) \\[0.5em]
(w_{1,2}^{(2)}\cdot\delta_1^{(2)} + w_{2,2}^{(2)}\cdot\delta_2^{(2)} + w_{3,2}^{(2)}\cdot\delta_3^{(2)} + w_{4,2}^{(2)}\cdot\delta_4^{(2)})*f'(z_{2}^{(1)}) \\[0.5em]
\end{array} \right) = 
\left( \begin{array}{cccc}
\delta_{1}^{(1)} \\[0.5em]
\delta_{2}^{(1)} \\[0.5em]
\end{array}\right)
$$

We are now here in our neural network:


In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=1,
    special_inter_layer_arrow_offset="layer",
    bias=True
)

nn_plot.show()

This means that these are the last set of weights we have to compute!

#### Step 2: Compute $\nabla_{W^{(1)}}$ and $\nabla_{b^{(1)}}$
Our update rule for this layer is $\nabla_{W^{(1)}}=\delta^{(1)} a^{(0)T}$ and for bias $\nabla_{b^{(1)}}=\delta^{(1)}$. Notice that $a^{(0)}$ corresponds to the input layer. Because they're just the inputs, and neural networks do not apply activations on inputs before going through the network. This means that
$$a^{(0)} = x = 
\left( \begin{array}{cccc}
x_{1} \\[0.5em]
x_{2} \\[0.5em]
\end{array}\right)$$

then our new weight gradient

$$
\Large\nabla_{W^{(1)}}\normalsize = 
\left( \begin{array}{cccc}
\delta_1^{(1)}\\[0.5em]
\delta_2^{(1)}\\[0.5em]
\end{array}\right)
\left( \begin{array}{cccc}
x_{1} \\[0.5em]
x_{2} \\[0.5em]
\end{array}\right)^T = 
$$
$$
\left( \begin{array}{cccc}
\delta_1^{(1)}\\[0.5em]
\delta_2^{(1)}\\[0.5em]
\end{array}\right)
\left( \begin{array}{cccc}
x_{1} & x_{2}\\[0.5em]
\end{array}\right) = 
\left( \begin{array}{cccc}
\delta_1^{(1)}x_1 &\delta_1^{(1)}x_2\\[0.5em]
\delta_2^{(1)}x_1 &\delta_2^{(1)}x_2 \\[0.5em]
\end{array}\right)
$$

our new bias gradient
$$\Large \nabla_{b^{(1)}}=\delta^{(1)}\normalsize = \left( \begin{array}{cccc}
\delta_1^{(1)}\\[0.5em]
\delta_2^{(1)}\\[0.5em]
\end{array}\right)$$ 

#### Step 3: We perform our final gradient for the sample

for weights:
$$
\Large W^{(1)} \normalsize  =\left( \begin{array}{cc}
w_{1,1}^{(1)} & w_{1,2}^{(1)} \\[0.5em]
w_{2,1}^{(1)} & w_{2,2}^{(1)} \\[0.5em]
\end{array}\right)
$$
$$
\Large \nabla{W^{(1)}} \normalsize = \left( \begin{array}{cccc}
\delta_1^{(1)}x_1 &\delta_1^{(1)}x_2\\[0.5em]
\delta_2^{(1)}x_1 &\delta_2^{(1)}x_2 \\[0.5em]
\end{array}\right)
$$
for biases
$$
\Large b^{(1)} \normalsize  = \left( \begin{array}{cc}
b_{1}^{(1)}\\[0.5em]
b_{2}^{(1)}\\[0.5em]
\end{array}\right)
$$
$$
\Large \nabla_{b^{(1)}}=\delta^{(1)}\normalsize = \left( \begin{array}{cccc}
\delta_1^{(1)}\\[0.5em]
\delta_2^{(1)}\\[0.5em]
\end{array}\right)
$$

Gradient Descent:

$$
W_{new}^{(1)} = \left( \begin{array}{cc}
w_{1,1}^{(1)} & w_{1,2}^{(1)} \\[0.5em]
w_{2,1}^{(1)} & w_{2,2}^{(1)} \\[0.5em]
\end{array}\right) - \eta\left( \begin{array}{cccc}
\delta_1^{(1)}x_1 &\delta_1^{(1)}x_2\\[0.5em]
\delta_2^{(1)}x_1 &\delta_2^{(1)}x_2 \\[0.5em]
\end{array}\right)
$$

$$
b_{new}^{(1)} = 
\left( \begin{array}{cc}
b_{1}^{(1)}\\[0.5em]
b_{2}^{(1)}\\[0.5em]
\end{array}\right) - \eta\left( \begin{array}{cccc}
\delta_1^{(1)}\\[0.5em]
\delta_2^{(1)}\\[0.5em]
\end{array}\right)
$$

We have now finished backpropagation:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=0,
    special_inter_layer_arrow_offset="node",
    bias=True
)

nn_plot.show()

### Closing Remarks on Backpropagation: Tips and tricks

#### For regression
If you want to do the same with regression, your last weight layer will simply be a 4x1 matrix instead, with no activation. This means that your loss $\delta^{(3)}$ would simply be 1 value, $\hat{y}-\bar{y}$, which makes sense because you would only have 1 node, essentially a 1x1 matrix. Make sure you line up the matrices perfectly for this case. I highly recommend trying these problems out with concrete numbers, as I will only go over theory here. Feed problems into ChatGPT, and ask for small regression or classification networks with 1 or 2 hidden layers to feed forward and backpropagate through

#### A caveat for ReLU

ReLU is a weird function in such that it is "technically" non-differentiable. While however, with coding neural networks, we defined ReLU and its derivative to be simply 0 at the cusp, so that we maintain some sort of differentiability. We can do this by exploiting if else statements, like that of ```np.where()``` in the numpy package.



#### A common mistake
People generally tend to mix up the activation function and its derivative. Remember that for forward pass, we use the activation function, and backwards pass, when computing the loss we use the derivative of the activation function. This might be easier to understand if we visualize it. Let's come up with some synthetic values for our previous nueral network for the following layer:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="layer",
    bias=True
)

nn_plot.show()

Recall ReLU:
$$
\operatorname{ReLU}(x) = \begin{cases}
x, & \text{if } x \geq 0, \\
0, & \text{if } x < 0.
\end{cases}
$$
$$
\operatorname{ReLU'}(x) = \begin{cases}
1, & \text{if } x \geq 0, \\
0, & \text{if } x < 0.
\end{cases}
$$

lets assume we calculated $z^{(2)}$ to be:

$$
z^{(2)} = 
\left( \begin{array}{cc}
1.35\\[0.5em]
-4.30\\[0.5em]
6.78\\[0.5em]
-.32\\[0.5em]
\end{array}\right)
$$
then  $a^{(1)} =  f(z^{(2)}) = $
$$
\left( \begin{array}{cc}
1.35\\[0.5em]
0\\[0.5em]
6.78\\[0.5em]
0\\[0.5em]
\end{array}\right)
$$

__**It is extremely important to understand that $a^{(1)} \neq f'(z^{(1)})$. Remember that $a^{(1)} = f(z^{(1)})$. You can not use the derivative of $z^{(1)}$ to compute $z^{(1)}$. This is another extremely common mistake**__ This is how they are different:

$$
\text{ where } z^{(2)} = 
\left( \begin{array}{cc}
1.35\\[0.5em]
-4.30\\[0.5em]
6.78\\[0.5em]
-.32\\[0.5em]
\end{array}\right)
$$


our forward pass (activated values for nodes 1, 2, 3, and 4 would be as such)
$$
f(z^{(2)}) = \left( \begin{array}{cc}
1.35\\[0.5em]
0\\[0.5em]
6.78\\[0.5em]
0\\[0.5em]
\end{array}\right)
$$

Notice the difference:
$$
f'(z^{(2)}) = \left( \begin{array}{cc}
1\\[0.5em]
0\\[0.5em]
1\\[0.5em]
0\\[0.5em]
\end{array}\right)
$$

Let's look at our backpropagation example. with $z^{(2)}$ This means we would be located here in our network:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2, 4, 3,3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_target_layer=2,
    special_inter_layer_arrow_offset="node",
    bias=True
)

nn_plot.show()

Keep this in mind. You use $f(z^{(2)})$ in forward pass and $f'(z^{(2)})$ in backpropagation. This goes for all of the other differentiable activations too. When we compute the hadamard product when doing our backpropagation example, you see how this becomes


$$\Large \delta^{(2)}\normalsize =
\left(\begin{array}{cccc}
(w_{1,1}^{(3)}\delta_1^{(3)} + w_{2,1}^{(3)}\delta_2^{(3)} + w_{3,1}^{(3)}\delta_3^{(3)})*f'(z_{1}^{(2)})\\[0.5em]
(w_{1,2}^{(3)}\delta_1^{(3)} + w_{2,2}^{(3)}\delta_2^{(3)} + w_{3,2}^{(3)}\delta_3^{(3)})*f'(z_{2}^{(2)}) \\[0.5em]
(w_{1,3}^{(3)}\delta_1^{(3)} + w_{2,3}^{(3)}\delta_2^{(3)} + w_{3,3}^{(3)}\delta_3^{(3)})*f'(z_{3}^{(2)})\\[0.5em]
(w_{1,4}^{(3)}\delta_1^{(3)} + w_{2,4}^{(3)}\delta_2^{(3)} + w_{3,4}^{(3)}\delta_3^{(3)})*f'(z_{4}^{(2)}) \\[0.5em]
\end{array}\right) = 
$$
$$
\left(\begin{array}{cccc}
(w_{1,1}^{(3)}\delta_1^{(3)} + w_{2,1}^{(3)}\delta_2^{(3)} + w_{3,1}^{(3)}\delta_3^{(3)})*1\\[0.5em]
(w_{1,2}^{(3)}\delta_1^{(3)} + w_{2,2}^{(3)}\delta_2^{(3)} + w_{3,2}^{(3)}\delta_3^{(3)})*0 \\[0.5em]
(w_{1,3}^{(3)}\delta_1^{(3)} + w_{2,3}^{(3)}\delta_2^{(3)} + w_{3,3}^{(3)}\delta_3^{(3)})*1\\[0.5em]
(w_{1,4}^{(3)}\delta_1^{(3)} + w_{2,4}^{(3)}\delta_2^{(3)} + w_{3,4}^{(3)}\delta_3^{(3)})*0 \\[0.5em]
\end{array}\right) $$

Feel free to refer back here, as you may make this mistake in the future.

## From Matrices to Tensors: The Need for Batches:
---

The example that we had just performed forward pass and backpropagation on was for but one sample in our dataset. For the following, we can model our neural network again, to show you what I mean:

In [ ]:
nn_vis = NeuralNetworkVisualization(batchSize=1, nodes_per_layer=[2,2,4,3,3],bias=True)

**Tip:** For this model, drag with your cursor to see a 3d view.

You can see we have 2 inputs, and then 3 possible categories to choose from. This is all for one sample in a dataset that may contain millions. We essentially performed **stochastic gradient descent.** The intuition is that every single sample individually, then use that to step in the direction of the local minima. This is called stochastic gradient descent. Meanwhile, normal gradient descent is when you compute the gradient with to EVERY SINGLE SAMPLE AT THE SAME TIME, then averaging them. For a dataset with a million datapoints, and neural networks with 100+ nodes per layer, you see how inefficient this becomes (you'd brick your computer!). 

However, something to note is that every sample might deviate a little bit from its idealistic prediction. This introduces noise when performing gradient descent, and as a result we do not travel directly into the local minima. Let's look at the following example:

In [ ]:
sim = GradientDescentSimulator(learning_rate=10, title="Stochastic Gradient Descent",max_iter=20,noise_level=.05,random_seed=99)
sim.fig.show()

You see the issue here? There is a lot of noise when we are trying to compute the gradient, since one sample, while it can roughly approximate the gradient for every batch, will add noise to every step. We then "deviate a little bit", so that we can never reach the bottom of the local minima. So how do we solve this problem?
### Solution: batched gradient descent (hyperparameter 2)

Let's calculate maybe only 10 samples at the same time, average them, and then update the weights accordingly. We can build a neural network that looks like this:

In [ ]:
nn_vis = NeuralNetworkVisualization(batchSize=10, nodes_per_layer=[2,2,4,3,3],bias=True)

You see how now, we can input 10 samples of our 2 feature dataset at a time? We then average them, and then compute the gradients with respect to the batch. We essentially have 10 neural networks running in parallel, which we will call a "slice" for the purpose of this demonstration.

Let's let $W$ denote all of the weights and biases for every layer in a neural network "slice".

 **Here, the values for $W$ will be the same for every single slice**. However, when we compute the gradients $\nabla W$, notice that all of the samples are different across slices, and so each "slice" will have a different gradient for their layers of weights and biases. So across all of the batches, **we average $\nabla W$**, so that we only have one set of weights for our batched neural network. We use this as our new gradient to update the weights with respect to the first layer. Then, these new weights for the first layer are all used to "update" the weights for all the other layers, so that the weights and biases for every neural network slice match again. Look what happens when we do this:


In [ ]:
sim = GradientDescentSimulator(learning_rate=10, title="Batched Gradient Descent",max_iter=100,noise_level=.01,tolerance=.1,random_seed=99)
sim.fig.show()

We still have a little noise, but the convergence is much better, and we can process our dataset much quicker. In practice, these "tensors" are how computers actually operate neural networks. Whenever you talk about specifying a batch size, this is generally what you are doing. Please note that each dataset might need different batch sizes based on their size. For example a dataset of 100 samples may not need batches, while a dataset of 1 million samples definitely will. Play around with it and see what gives you the best convergence for your dataset.

## Introduction to Different Optimizers (hyperparameter 4)
---

You have seen how we can use gradient descent in order to take steps into the local minima. Gradient Descent is a type of **optimizer**.

However, it turns out that neural networks spaces, with their multiple layers and their non-linear activation functions end up perverting the objective loss function such that it is no longer just a nice bowl, like we have seen in the previous examples. It ends up looking more like a ruggedy terrain, with lots of little dips and mountains. As a result what ends up happening is that our ball will end up getting "stuck" in a local minima, and never achieve better generalization. Let's look what happens in a scenario like this:

In [ ]:
sim_multi = MultiMinimaSimulator(learning_rate=1, title="Learning Rate that is optimized for the parameter space",camera_distance=1,azim=135,elev=25)
sim_multi.fig.show()


As you can see, our ball travels down into one of the two dips, but you can see that the other one is a lot deeper. Having our ball go down there, we would get a model that can effectively generalize better. So naively we can try turning up the learning rate and see if we can "jump" into the better minima:

In [ ]:
sim_multi = MultiMinimaSimulator(learning_rate=5, title="Learning Rate that is too High for a Non-convex Space",camera_distance=1,azim=35,elev=25)
sim_multi.fig.show()

We can see that we do jump over the first minima, but the sides of the objective function are much steeper. As a result the gradient tells us to "throw the ball really hard," pushing us back out of the local minima dip. We then just jump around the minimas and never converge. So how do we combat this?

### Different optimizers to combat this issue

#### An important Caveat 
Before going on to cover these algorithms, all of these gradient and learning rate updates are with respect to the iterations of each individual layer's gradients. This means that each new hyperparameter we add that is not $\eta$ or the gradient $\nabla W$ will also have the superscript $(\ell)$, denoting it's respect to its gradient. This is because all layers' gradients are seperate with respect to each other (note this is different from being independent). All of these optimizers are recursive in nature, and work on there previous individual iterations. This means we will update for as many batches that we have to divide out datset. The optimizer methods we are about to cover update with respect to these batches. The gradients are averaged per batch, and we use this averaged gradient as our update rule. **This is called layer-wise dependency**

#### AdaGrad

There have been many algorithms to combat this issue, and help converge at a better solution. The first of these is AdaGrad. Do you remember how we had to tune the learning rate ourselves? Well what if we made an algorithm that also helped tune the learning rate by accumulating these gradients? This is what AdaGrad proposes, and it's following algorithm can be modeled as such:

$$W_{new} = W - \frac{\eta}{\sqrt{G_t}+\epsilon}g_t$$

where
 - $g_t$ is the current gradient at the current iteration $t$, equivalent to $\nabla W$, representing the gradient for every individual layer. Note that this is not additive; it is more like a set.
 - $G_t$ is the sum of the sqare of all previous gradients, or $\large \sum_{i=1}^t g_i^2$.  For a loop, you can think of this as $G = G + (g_t)^2$
 - t is batch t out of how ever many batches it takes to subdivide the whole dataset
 - $\epsilon$ is usually some super small constant (usually $1*10^{-8}$) used to ensure no divide by 0 error
 - $\eta$ is the learning rate
 - $W_{new}$ is the new weights for all the layers of the matrix
 - $W$ is the weights for all the layers of the matrix

**Note:** This means that for our update rule, for every iteration, you "save" $g_t^{(\ell)}$'s weights and accumulate the square of them for $G_t^{(\ell)}$. You will therefore update $G_t^{(\ell)}$ recursively, usually via a loop.

What this essentially does is that for gradients (layers in our case) with high gradients and frequent big updates, we slow down in those directions, while maintaining speed for our sparser gradient updates, in the directions where the objective function isn't as steep. With this we solve the problem of the averages of the gradients not pushing us enough in some directions while pushin us too much in others, leading to poorer convergence in normal or stochastic gradient descent.
By keeping track of the square of the gradients for each layer, we can kind of "adapt" the learning rate depending on the slope. As an analogy, imagine you're in a car, driving on a city road with a lot of bumps and potholes. With AdaGrad, your car adjusts its speed by taking into account every pothole you’ve encountered from the start of your trip.

This way, we slowly start to "slow down" as we encounter minima. We start with a fast initial learning rate, but then it slowly diminishes, coming to zero. Going back to our analogy, over time, because you remember all the rough spots, your speed gets dialed down too much; even in areas where the road is relatively smooth—making your journey unnecessarily slow. This means you may just end up coming to a stop completely before reaching a good minimum. 

#### RMSProp

RMSProp aims to solve this issue where the gradients diminish too quickly by taking the moving average of the adaptive learning rates. This means that our older gradients decay, and our newer gradients will contribute more to the moving average, ensuring that the update reflects more recent information. The following algorithm looks as such: 

$$v_t = \beta v_{t-1} + (1-\beta)g_t^2$$
$$W_{new} = W - \frac{\eta}{\sqrt{v_t}+\epsilon}g_t$$
$$v_0 = 0$$

where
 - $\beta$ is the decay rate. It is usually set at .9
 - $v_t$ is the moving average of squared gradients at batch t. 
 - $g_t$ is the current gradient at the current iteration $t$, equivalent to $\nabla W$, representing the gradient for every individual layer. Note that this is not additive; it is more like a set, so you would have seperate $v_t$'s for each layer, or a $v_t^{(\ell)}$
 - t is batch t out of how ever many batches it takes to subdivide the whole dataset
 - $\epsilon$ is usually some super small constant (usually $1*10^{-8}$) used to ensure no divide by 0 error
 - $\eta$ is the learning rate
 - $W_{new}$ is the new weights for all the layers of the matrix
 - $W$ is the weights for all the layers of the matrix

**Note:** This means that for our update rule, for every iteration, you "save" $v_t^{(\ell)}$ as it's own seperate entity per layer of weights. $v_t{(\ell)}$ will have the same dimensions as your weight matrix. You use the update rule above to get $v_t{(\ell)}$ for the next batch.

This solves the problem where the accumulated gradients were slowing down the optimization well before it reached it local minima. As for the continuation of our analogy:

With RMSProp, your car only pays attention to the potholes you’ve seen in the recent part of your drive. This means your speed is adjusted based on the current road conditions rather than being weighed down by every past bump. You’re still cautious, but not overly so, and you can keep a steadier pace overall.

While RMSProp is extremely powerful, it still has the potential to get stuck in local minima, because the squared gradients peter out when we reach some local minima. This means we will converge in the local minima, while there may be a better minima over the horizon. ie. We can still get stuck in potholes before we get to our destination! Also notice that for $v_0$, we essentially get that our adaptive learning rate needs to "adjust" from 0 in order to get the best moving average for the scenario. This is equivalent to the need to start up your car. Time is precious, and you don't want to waste it!

#### Adam

Adam aims to solve RMSProp's convergence in smaller local minimums by adding first order momentum to the algorithm, and also a bias correction, so we dont need to "start up our car," as I had put it previously. Adam updates can be defined with the following formulas:

$$m_t = \beta_1 m_{t-1} + (1-\beta_1)g_t$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2)g_t^2$$
$$\text{Bias Correction: } \hat{m_t} = \frac{m_t}{(1-\beta_1^t)} \text {. The superscript t represents an exponent per iteration}$$
$$\text{Bias Correction: } \hat{v_t} = \frac{v_t}{(1-\beta_2^t)} \text {. The superscript t represents an exponent per iteration}$$
$$W_{new} = W - \eta\frac{\hat{m_t}}{\sqrt{\hat{v_t}}+\epsilon}g_t$$
$$m_0 = 0$$
$$v_0 = 0$$

**Note:** This means that for our update rule, for every iteration, you "save" $v_t^{(\ell)}$ and $m_t^{(\ell)}$ as it's own seperate entity per layer of weights. $v_t{(\ell)}$ and $m_t^{(\ell)}$ will have the same dimensions as your weight matrix. You use the update rule above to get $v_t{(\ell)}$ and $m_t^{(\ell)}$, apply the bias correction seperately, then use the uncorrected $v_t{(\ell)}$ and $m_t^{(\ell)}$ for the next batch. **This was a common source of confusion for me when coding adam myself. When you are updating the first and second moments, you should only bias correct the value right before performing a step of gradient descent. The m and v should stay seperate from this bias correction when you recursively update them.**

where
 - $\beta_1$ is the decay rate for momentum. It is usually set at .9
 - $\beta_2$ is the decay rate for squared gradient moving average (2nd moment). It is usually set at .999
 - $v_t$ is the moving average of squared gradients at batch t. 
 - $m_t$ is the momentum of our function.
 - $g_t$ is the current gradient at the current iteration $t$, equivalent to $\nabla W$, representing the gradient for every individual layer. Note that this is not additive; it is more like a set, so you would have seperate $v_t$'s and $m_t$'s for each layer, or a $v_t^{(\ell)}$ and $m_t^{(\ell)}$
 - t is batch t out of how ever many batches it takes to subdivide the whole dataset
 - $\epsilon$ is usually some super small constant (usually $1*10^{-8}$) used to ensure no divide by 0 error
 - $\eta$ is the learning rate
 - $W_{new}$ is the new weights for all the layers of the matrix
 - $W$ is the weights for all the layers of the matrix

With this newly added "momentum," we essentially press our accelerator in these "potholes," and this gives us enough power to drive out of them, all the way up until we reach our desired destination! While thinking of these analagies, look at the simulation below:


In [ ]:
sim_adaptive = MultiMinimaSimulatorAdaptive(
    learning_rate=0.7,
    title="Optimizer Comparisons",
    azim=50,  
    elev=35,   
    r_cam=1  
)

sim_adaptive.fig.show()

You can clearly see why I opted to use Adam in my neural network implementation. However, I do not want you to discourage you from using any of these, as all optimization problems will be different, and you should choose the best optimizer that suits your specific problem!

#### AdamW (hyperparameter 5)
If you have seen logistic regression, you are probably familiar with the term regularization. There are many different types of regularization, such as $\ell_1$ (LASSO) regularization, which uses the absolute value of the weights to create a sparser solution, or more commonly $\ell_2$ (Ridge) Regression, which is the square of the weights used for regularization. Notice there are many other types of regularization such as ElasticNet, or even regularizations with powers $p<1$. The issue with these regularization techniques, however, is that the gradients are updated adaptively, meaning the steps we take will change over time, and will not directly correlate to the size of the weights in any given layer. This means that some layers can have growing gradients, some can have shrinking gradients, and some may have very sparse gradients (matrices with a lot of 0's), and as a result, we get inconsistent regularization. This is where AdamW comes in. AdamW can be defined as
$$W_{new} = W - \eta(\frac{\hat{m_t}}{\sqrt{\hat{v_t}}+\epsilon}g_t + \lambda W)$$

where are new term
 - $\lambda$ is the hyperparameter specified by the client for how strong they want their decay

As opposed to $\ell_2$ regularization:
$$W_{new} = W - \eta\frac{\hat{m_t}}{\sqrt{\hat{v_t}}+\epsilon}g_t$$
$$g_t = \nabla W + \lambda W$$
You can see that $\ell_2$ regularization is encapsulated inside the gradient, which can pose the issue of this inconistent scaling, since the adaptive learning will distort the $\ell_2$ example. What adamW (the first example) does is it "decouples" the regularization, so that way the regularization scales independently of the adaptive learning rates and momentum calculations, and the regularization becomes consistent. **This is why if you are performing any sort of norm regularization, you should use AdamW to ensure consistent regularization.**



## Other Regularization Techniques
---

We have covered 1 regularization technique. However, it is important to have multiple different types of regularization techniques, because each technique may help with certain aspects of your model overfitting. For example, we don't want any couple of nodes predicting using only one feature for its predictions, so we apply weight regularization, so that each feature and weight has more opportunity to learn, and therefore increase generalizeability. Our second types of regularization comes in the form of dropout:

### Dropout (hyperparameter 6):
The idea behind this is that we randomly drop some nodes in a layer per batch so that our neural network can not rely on any one given node, which should increase generalizability. While we are still not sure how it works, it still proves to be an extremely effective regularization technique. Here is an example picture: 


In [ ]:
display(Image(filename='Visualizers/dropout.png'))

#### Applying dropout
Our dropouts can be defined by 
$$X \sim Binomial(n,(1-p))$$

where 
 - p is the probability specified by the client
 - n is the number of nodes in the layer

What we do is we essentially create a mask, or some random probability distribution of 0's depending on the specified probability. Let's revisit an old example, where the arrow is pointing to a layer we want to apply dropout to:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2,4, 3, 3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_offset="node",
    special_inter_layer_arrow_target_layer=2
)
# Display the plot.
nn_plot.show()

Then n = 4, and let's assume we have a dropout of .2. We generate a list of size n with 80$ of generating a 1 and 20% chance of generating a zero. We could hypothetically generate something like this:
$$
\left( \begin{array}{ccc}
1\\[0.5em]
0 \\[0.5em]
1\\[0.5em]
1
\end{array}\right)
$$

Note that this could be different every single time.

Then, in order to balance out our lost weights, we multiply our "mask" by $\frac{1}{(1-p)}$:
$$
\frac{1}{(1-.2)}
\left( \begin{array}{ccc}
1\\[0.5em]
0 \\[0.5em]
1\\[0.5em]
1
\end{array}\right) = 
\left( \begin{array}{ccc}
1.25\\[0.5em]
0 \\[0.5em]
1.25\\[0.5em]
1.25
\end{array}\right)
$$
Then our layer with dropout would be:
$$\left( \begin{array}{ccc}
1.25*w_{1,1}^{(3)} & 1.25*w_{2,1}^{(3)} & 1.25*w_{3,1}^{(3)} \\[0.5em]
0 & 0& 0\\[0.5em]
1.25*w_{1,3}^{(3)} & 1.25*w_{2,3}^{(3)} & 1.25*w_{3,3}^{(3)} \\[0.5em]
1.25*w_{1,4}^{(3)} & 1.25*w_{2,4}^{(3)} &1.25* w_{3,4}^{(3)}
\end{array}\right)$$

This would last up until we compute everything in the batch and backpropagate correctly. Then we would do the same thing for the next batch, but notice the placement of 0's and 1's might be different, because they are picked randomly from a binomial distribution.

#### Caveat: What if the dropout drops all nodes?
While you may be thinking of this, it can be a good edge case to handle. However, this is an extremely rare case, because you would eseentially have to multiply (1-dropout rate) * number of nodes, which for large layers would give next to 0 probability. However, in the rare event you get a mask with all 0's you can just flip the top one to 1, since this is so statistically improbabable it will most likely not hurt your generalization. You can pick a random 1 to flip, but note that any computational expense added in backpropagation, training, or anything with loops will blow up time complexity.

### Input Dropout (Hyperparameter 7)

Likewise, we can apply the same thing to the input layer, that way our neural network can't ever rely on any one such feature.

### Closing Notes: Weight Initializations

If you are building a neural network yourself, I will save you a HUGE headache. This took me 9 hours to figure out when coding my weights and trying to figure out why my models were never converging.

In theory, weights in all of the weight matrices can be initialized randomly, but in practice, this is actually horrible. If we do not standardize our weights, then our model might contain some sort of symmetry, or have some sort of pattern in certain parts of the weight matrices that can make gradients explode or vanish. For example, if we have a weight and it's the first layer, that means we chain together the derivatives of all the other layers, all of these chained derivatives can make our value explode, or vanish, if our initial values isn't set up yet.

#### Common Initialization tecnhiques

The method I used is called He initialization which utilizes a normal distribution to initialize the weights. It's most popular for ReLU, but works fairly well for sigmoid and tanh as well. Here is the following formula:

$$N \sim(0, \frac{2}{\text{nodes of the previous layer}} )$$

The formula for for Xavier/Gloat Initialization is very similar. This works well for tanh and sigmoid:
$$N \sim(0, \frac{1}{\text{nodes of the previous layer}} )$$

For biases, it is common to just set these to 0. This is done because the model should "adjust the height" with every backpropagation step.
Given the picture below:

In [ ]:
nn_plot = NeuralNetworkPlot(
    nodes_per_layer=[2, 2,1, 3, 3],
    label_arrows=False,
    arrow_label_overrides={},
    special_inter_layer_arrow_color="blue",
    special_inter_layer_arrow_offset="node",
    special_inter_layer_arrow_target_layer=2
)
# Display the plot.
nn_plot.show()

our nodes of the previous layer for every single node will be 2!
That means we would pick a value from that distribution that we described, and make that the weight of our node.

**WARNING!!!!!!!!!!!!!!!!! DO NOT FORGET THIS. I ALMOST WENT INSANE TRYING TO FIX THIS.**

Unless you want to spend 9 hours debugging :))))))))))))))))))))))))))))))))))))))))))

## Conclusion
---
Ths is the end of my document. We went over the history and reasons behind using neural networks, the forward and backwards pass, optimizers, hyperparameters, gradient descent, batches, and regularization techniques. You should be ready with all of this information to build your own neural network from scratch! Feel free to use this document as a reference for building your own neural network! I will have the listed derivaations from earlier in the document below this section for anyone who is interested in the math and derivations behind the loss, and step-by-step breakdowns on the derivatives of the activation and loss functions, as well as any other fun facts. Thank you so much for reading!